In [2]:
import numpy as np
import random
from itertools import permutations
from typing import List, Tuple, Optional, Callable
import time
import pulp
import copy
import yaml
from itertools import combinations
from CBS import load_map, get_random_free_position
from Final_CBS import Environment, CBS

In [3]:
# NOTE: add flag for assigned or not
class CapabilityProfile:
    def __init__(self,
                 robot_id: str,
                 mobility_type: str,
                #  max_speed: float,
                 max_speed: int, # keep speed a constant integer of 1 for CBS purposes for now
                 payload_capacity: Optional[float],
                 reach: Optional[float],
                 battery_life: float,
                 size: Tuple[float, float, float],
                 environmental_resistance: Optional[List[str]],
                 sensors: Optional[List[str]],
                 sensor_range: Optional[float],
                 manipulators: Optional[List[str]],
                 communication_protocols: List[str],
                 processing_power: float,
                 autonomy_level: str,
                 special_functions: Optional[List[str]],
                 safety_features: Optional[List[str]],
                 adaptability: bool,
                 location: Tuple[float, float, float],
                 #preferred_tasks: List[str],
                 current_task: Optional["TaskDescription"],
                 remaining_distance: float,
                 time_on_task: float,
                 tasks_attempted: int,
                 tasks_successful: int,
                 current_path: list, # the path the robot is currently following, if any
                 assigned: bool): # Indicates if the robot is currently assigned to a task
        self.robot_id = robot_id
        self.mobility_type = mobility_type
        self.max_speed = max_speed
        self.payload_capacity = payload_capacity
        self.reach = reach
        self.battery_life = battery_life
        self.size = size  # (length, width, height)
        self.environmental_resistance = environmental_resistance
        self.sensors = sensors
        self.sensor_range = sensor_range
        self.manipulators = manipulators
        self.communication_protocols = communication_protocols
        self.processing_power = processing_power
        self.autonomy_level = autonomy_level
        self.special_functions = special_functions
        self.safety_features = safety_features
        self.adaptability = adaptability
        self.location = location  # (x, y, z) coordinates
        #self.preferred_tasks = preferred_tasks
        self.current_task = current_task  # The task the robot is currently assigned to
        self.remaining_distance = remaining_distance  # Distance left to the current task location
        self.time_on_task = time_on_task  # Time spent on the current task
        self.tasks_attempted = tasks_attempted
        self.tasks_successful = tasks_successful
        self.current_path = current_path
        self.assigned = assigned # says if the robot is assigned to a task to replace the index lists below

    def __repr__(self):
        return (f"CapabilityProfile(robot_id={self.robot_id}, mobility_type={self.mobility_type}, "
                f"max_speed={self.max_speed}, payload_capacity={self.payload_capacity}, "
                f"reach={self.reach}, battery_life={self.battery_life}, size={self.size}, "
                f"environmental_resistance={self.environmental_resistance}, sensors={self.sensors}, "
                f"sensor_range={self.sensor_range}, manipulators={self.manipulators}, communication_protocols={self.communication_protocols}, "
                f"processing_power={self.processing_power}, autonomy_level={self.autonomy_level}, "
                f"special_functions={self.special_functions}, safety_features={self.safety_features}, "
                f"adaptability={self.adaptability}, location={self.location}, remaining_distance={self.remaining_distance}, time_on_task={self.time_on_task}, tasks_attempted={self.tasks_attempted}, tasks_successful={self.tasks_successful})")

class TaskDescription:
    def __init__(self,
                 task_id: str,
                 task_type: str,
                 objective: str,
                 priority_level: str,
                 reward: float,
                 difficulty: float,
                 location: Tuple[float, float, float],
                 navigation_constraints: Optional[List[str]],
                 required_capabilities: List[str],
                 time_window: Optional[Tuple[str, str]],
                 environmental_conditions: Optional[List[str]],
                 dependencies: Optional[List[str]],
                 tools_needed: Optional[List[List[str]]],
                 communication_requirements: Optional[List[str]],
                 safety_protocols: Optional[List[str]],
                 performance_metrics: List[str],
                 success_criteria: str,
                 assigned_robot: Optional["CapabilityProfile"],
                 time_to_complete: float,
                 assigned: bool):
        self.task_id = task_id
        if (task_id in {"maintenance", "cleaning"}): #figure out tasks that won't have progress reset after interruption
            self.reset_progress = False 
        else:
            self.reset_progress = True
        self.task_type = task_type
        self.objective = objective
        self.priority_level = priority_level
        self.reward = reward
        self.difficulty = difficulty
        self.location = location  # (x, y, z) coordinates
        self.navigation_constraints = navigation_constraints
        self.required_capabilities = required_capabilities
        self.time_window = time_window  # (start_time, end_time) or None
        self.environmental_conditions = environmental_conditions
        self.dependencies = dependencies
        self.tools_needed = tools_needed
        self.communication_requirements = communication_requirements
        self.safety_protocols = safety_protocols
        self.performance_metrics = performance_metrics
        self.success_criteria = success_criteria
        self.assigned_robot = assigned_robot  # The robot currently assigned to this task
        self.time_to_complete = time_to_complete  # Total time required to complete the task
        self.time_left = time_to_complete # Counter used during the step method, resets back to time_to_complete based on reset_progress
        self.assigned = assigned

    def __repr__(self):
        return (f"TaskDescription(task_id={self.task_id}, task_type={self.task_type}, "
                f"objective={self.objective}, priority_level={self.priority_level}, "
                f"location={self.location}, reward={self.reward}, difficulty={self.difficulty}, navigation_constraints={self.navigation_constraints}, "
                f"required_capabilities={self.required_capabilities}, time_window={self.time_window}, "
                f"environmental_conditions={self.environmental_conditions}, "
                f"dependencies={self.dependencies}, tools_needed={self.tools_needed}, "
                f"communication_requirements={self.communication_requirements}, "
                f"safety_protocols={self.safety_protocols}, performance_metrics={self.performance_metrics}, "
                f"success_criteria={self.success_criteria}, time_to_complete={self.time_to_complete}), time_left={self.time_left}) ")

In [4]:
MOBILITY_TYPES = ["wheeled", "tracked", "legged", "aerial", "hovering", "climbing"]
ENV_RESISTANCES = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant", "wind-resistant"]
SENSORS = ["camera", "microphone", "LiDAR", "GPS", "ultrasonic", "temperature sensor", "infrared", "proximity sensor", "magnetometer", "ultrasonic"]
MANIPULATORS = ["gripper", "drill", "welding tool", "hydraulic bucket", "dispenser", "mixer drum", "cable hoist"]
COMM_PROTOCOLS = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
SPECIAL_FUNCS = ["object recognition", "speech output", "facial recognition", "object tracking", "gesture recognition",
                 "precise alignment", "terrain leveling", "balance control", "concrete mixing", "object detection"]
SAFETY_FEATS = ["emergency stop", "collision avoidance", "overheat protection", "fall detection",
                "obstacle detection", "speed reduction in crowded areas", "overload protection", "balance control"]

TASK_TYPES_RANDOM = ["delivery", "inspection", "cleaning", "monitoring", "maintenance", "assembly", "surveying", "data collection", "assistance"]
PRIORITIES = ["low", "medium", "high", "urgent"]
NAV_CONSTRAINTS = ["elevator", "stairs", "shelves", "no loud noises allowed", "narrow spaces", "low ceilings", "uneven floors",
                   "low visibility", "slippery", "crowded", "loose debris", "no-fly zone", "windy", "dense obstructions", "smooth surfaces"]
ENV_CONDITIONS = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant", "wind-resistant"]
PERF_METRICS = ["time taken", "accuracy", "energy consumption", "safety compliance", "completion rate"]

CAP_REQ_TEMPLATES = [
    "payload capacity >= 1.0",
    "payload capacity >= 5.0",
    "payload capacity >= 10.0",
    "payload capacity >= 20.0",
    "reach >= 1.5",
    "reach >= 3.0",
    "reach >= 5.0",
]

def _sample(items: List[str], k_min: int = 0, k_max: int = None) -> List[str]:
    """Sample between k_min and k_max items from the list."""
    if not items:
        return []
    if k_max is None:
        k_max = len(items)
    k_min = max(0, min(k_min, len(items)))
    k_max = max(k_min, min(k_max, len(items)))
    k = random.randint(k_min, k_max)
    return random.sample(items, k)

def _normalize_autonomy(level: str) -> str:
    if not level:
        return "semi-autonomous"
    s = level.strip().lower()
    if s in {"autonomous", "fully autonomous", "fully-autonomous", "full"}:
        return "fully autonomus"
    if "tele" in s:
        return "teleoperated"
    if "semi" in s:
        return "semi-autonomous"
    return "semi-autonomous"

def get_unique_task_location(tasks, grid, occupied_locations, max_attempts=100):
    """
    Finds a unique location for a new task that does not overlap with existing tasks or occupied robot locations.
    Occupied locations only include robot locations, not task locations so we check both with this function.
    Args:
        tasks: List of existing TaskDescription objects to check against.
        grid: The grid representing the environment, used to find free positions.
        occupied_locations: A set of tuples representing locations occupied by robots.
        max_attempts: Maximum number of attempts to find a unique location before giving up.

    Returns:
        location: A unique (x, y, z) location tuple for the new task.
    """
    attempts = 0
    task_locations = {task.location for task in tasks}
    all_occupied_locations = occupied_locations | task_locations  # Combine occupied locations with existing task locations
    location = get_random_free_position(grid, all_occupied_locations)
    # Ensure the location is unique and not occupied by any robot or existing task
    while any(np.allclose(location, occ_loc) for occ_loc in all_occupied_locations):
        attempts += 1
        if attempts >= max_attempts:
            raise Exception(f"Could not find a unique task location after {max_attempts} attempts")
        location = get_random_free_position(grid, all_occupied_locations)
    return location

In [5]:
def generate_random_robot_profile(robot_id: str, grid: List[List[int]], occupied_locations: set) -> CapabilityProfile:
    """Random robot with capability masking but all fields are always present.
    Lists are [] never None. Numeric fields exist even if 0."""

    # Base capabilities
    # I decided everything is going to have at least one random capability to increase suitability scores, the mask only
    # removes up to 3 capability groups but nothing that will cause immediate suitability zeroing
    mobility_type = random.choice(MOBILITY_TYPES)
    payload_capacity = round(random.uniform(20.0, 50.0), 1)
    reach = 0.0 if mobility_type in {"aerial"} else round(random.uniform(3.0, 10.0), 1)
    battery_life = round(random.uniform(grid.shape[0], grid.shape[0]*3), 1) # takes into account the size of the map
    size = (round(random.uniform(1.0, 5.0), 2), round(random.uniform(1.0, 5.0), 2), round(random.uniform(1.0, 5.0), 2))  # (length, width, height)
    environmental_resistance = _sample(ENV_RESISTANCES, 1)
    sensors = _sample(SENSORS, 1)  # should not be empty
    sensor_range = round(random.uniform(20.0, 50.0), 1) if sensors else 0.0
    manipulators = _sample(MANIPULATORS, 1) # should not be empty
    communication_protocols = _sample(COMM_PROTOCOLS, 1)  # should not be empty
    special_functions = _sample(SPECIAL_FUNCS, 1)
    safety_features = _sample(SAFETY_FEATS, 1)
    processing_power = round(random.uniform(5.0, 15.0), 1)
    autonomy_level = _normalize_autonomy(random.choice(["teleoperated", "semi-autonomous", "fully autonomous", "autonomous"]))
    adaptability = bool(random.getrandbits(1))

    # randomly mask 0-3 capability groups but keep types intact for suitability calculations
    groups = ["environmental_resistance", "special_functions", "safety_features"]
    to_mask = _sample(groups, 0, min(3, len(groups)))
    if "environmental_resistance" in to_mask: environmental_resistance = []
    if "special_functions" in to_mask: special_functions = []
    if "safety_features" in to_mask: safety_features = []

    return CapabilityProfile(
        robot_id=robot_id,
        mobility_type=mobility_type,
        max_speed=1.0, # CBS simplification
        payload_capacity=payload_capacity,
        reach=reach,
        battery_life=battery_life,
        size=size,
        environmental_resistance=environmental_resistance, # list
        sensors=sensors,
        sensor_range=sensor_range, # float (0 if no sensors)
        manipulators=manipulators,
        communication_protocols=communication_protocols,
        processing_power=processing_power,
        autonomy_level=autonomy_level,
        special_functions=special_functions,
        safety_features=safety_features,
        adaptability=adaptability,
        location=get_random_free_position(grid, occupied_locations),
        current_task=None,
        remaining_distance=0.0,
        time_on_task=0,
        tasks_attempted=0,
        tasks_successful=0,
        current_path=[],     # CBS path
        assigned=False
    )

def generate_random_task_description(task_id: str, grid: List[List[int]], occupied_locations: set, tasks) -> TaskDescription:
    task_type = random.choice(TASK_TYPES_RANDOM)
    # Map of task_type -> (tools_needed list, one required_cap string)
    task_domains = {
        "delivery": (["GPS"], "payload capacity >= 1.0"),
        "inspection": (["camera"], random.choice(["reach >= 1.5", "reach >= 3.0"])),
        "cleaning": (["gripper"], "payload capacity >= 5.0"),
        "monitoring": (["camera"], random.choice(["payload capacity >= 1.0", "payload capacity >= 5.0", "payload capacity >= 10.0"])),
        "maintenance": (["welding tool"], random.choice(["reach >= 1.5", "reach >= 3.0"])),
        "assembly": (["drill"], random.choice(["reach >= 1.5", "reach >= 3.0"])),
        "surveying": (["LiDAR"], random.choice(["payload capacity >= 1.0", "payload capacity >= 5.0"])),
        "data collection": (["temperature sensor"], random.choice(["payload capacity >= 1.0", "payload capacity >= 5.0"])),
        "assistance": (["microphone"], random.choice(["payload capacity >= 1.0", "payload capacity >= 5.0"]))
    }
    tools_needed, req_cap = task_domains.get(task_type, ([], None))
    required_caps = [req_cap] if req_cap else []

    # random tasks can have zero capability constraints to increase suitability scores
    nav = _sample(NAV_CONSTRAINTS, 0)
    env = _sample(ENV_CONDITIONS, 0)
    safety = _sample(SAFETY_FEATS, 0)
    comm = _sample(COMM_PROTOCOLS, 0)
    perf = random.choice(PERF_METRICS)

    duration = round(random.uniform(8.0, 20.0), 1)
    difficulty = round(random.uniform(1.0, 10.0), 1)
    reward = round(random.uniform(1.0, 10.0), 1)

    return TaskDescription(
        task_id=task_id,
        task_type=task_type,
        objective=f"Perform {task_type} task",
        priority_level=random.choice(PRIORITIES),
        reward=reward,
        difficulty=difficulty,
        location=get_unique_task_location(tasks, grid, occupied_locations),
        navigation_constraints=nav,
        required_capabilities=required_caps, # ALWAYS a list
        time_window=(f"{random.randint(8, 17)}:00", f"{random.randint(18, 23)}:00"),
        environmental_conditions=env,
        dependencies=[],
        tools_needed=list(tools_needed), # ALWAYS a list
        communication_requirements=comm,
        safety_protocols=safety,
        performance_metrics=perf, # keep single string for consistency
        success_criteria="Task completed within time window",
        assigned_robot=None,
        time_to_complete=duration,
        assigned=False
    )


In [23]:
#SAFETY_FEATS = ["emergency stop", "collision avoidance", "overheat protection", "fall detection",
#                "obstacle detection", "speed reduction in crowded areas", "overload protection", "balance control"]
STRICT_ROBOT_PROFILES = [
    {
        "name": "delivery",
        "mobility_type": "aerial",
        "environmental_resistance": ["weatherproof", "heat-resistant", "cold-resistant"],
        "sensors": ["LiDAR", "GPS", "proximity sensor", "camera"],
        "manipulators": ["gripper"],
        "communication_protocols": ["Wi-Fi", "4G"],
        "special_functions": ["object recognition", "object tracking", "facial recognition"],
        "safety_features": ["obstacle detection", "emergency stop"],
        "sensor_range": 25.0,
        "processing_power": 5.0,
        "autonomy_level": "fully autonomous",
        "payload_capacity": 20.0,
        "reach": 0.0,
        "battery_life": 80.0,
        "size": (2.0, 2.0, 2.0),
        "adaptability": False
    },
    {
        "name": "assembly",
        "mobility_type": "hovering",
        "environmental_resistance": ["dustproof", "heat-resistant", "shock-resistant"],
        "sensors": ["camera", "proximity sensor", "infrared"],
        "manipulators": ["gripper", "drill", "dispenser", "welding tool"],
        "communication_protocols": ["Wi-Fi", "Radio"],
        "special_functions": ["object recognition", "object tracking", "gesture recognition", "precise alignment"],
        "safety_features": ["collision avoidance", "overheat protection", "emergency stop"],
        "sensor_range": 10.0,
        "processing_power": 7.0,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 20.0,
        "reach": 5.0,
        "battery_life": 75.0,
        "size": (3.0, 2.0, 2.0),
        "adaptability": True
    },
    {
        "name": "excavator",
        "mobility_type": "tracked",
        "environmental_resistance": ["dustproof", "shock-resistant", "weatherproof"],
        "sensors": ["LiDAR", "camera", "proximity sensor", "magnetometer"],
        "manipulators": ["hydraulic bucket", "cable hoist"],
        "communication_protocols": ["Wi-Fi", "Radio"],
        "special_functions": ["terrain leveling", "object recognition"],
        "safety_features": ["collision avoidance", "overload protection", "emergency stop"],
        "sensor_range": 15.0,
        "processing_power": 8.0,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 20.0,
        "reach": 8.0,
        "battery_life": 75.0,
        "size": (4.2, 2.5, 3.0),
        "adaptability": True
    },
    {
        "name": "bricklayer",
        "mobility_type": "wheeled",
        "environmental_resistance": ["dustproof", "waterproof", "shock-resistant"],
        "sensors": ["infrared", "camera", "proximity sensor"],
        "manipulators": ["mixer drum", "dispenser", "gripper", "cable hoist"],
        "communication_protocols": ["Wi-Fi", "Radio"],
        "special_functions": ["precise alignment", "concrete mixing"],
        "safety_features": ["obstacle detection", "emergency stop", "collision avoidance"],
        "sensor_range": 10.0,
        "processing_power": 5.5,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 20.0,  
        "reach": 5.5,
        "battery_life": 80.0,
        "size": (4.0, 2.0, 2.0),
        "adaptability": False
    },
    {
        "name": "crane",
        "mobility_type": "tracked", 
        "environmental_resistance": ["weatherproof", "wind-resistant", "dustproof"],
        "sensors": ["camera", "GPS", "proximity sensor", "ultrasonic"],
        "manipulators": ["gripper", "cable hoist"],
        "communication_protocols": ["5G", "Radio"],
        "special_functions": ["precise alignment", "object tracking", "object detection"],
        "safety_features": ["overload protection", "balance control", "collision avoidance", "emergency stop"],
        "sensor_range": 30.0,
        "processing_power": 6.5,
        "autonomy_level": "semi-autonomous",
        "payload_capacity": 35.0,
        "reach": 30.0,
        "battery_life": 70.0,
        "size": (3.5, 3.5, 10.0),
        "adaptability": False
    },
    {
        "name": "scaffolding",
        "mobility_type": "climbing",
        "environmental_resistance": ["dustproof", "shock-resistant", "wind-resistant"],
        "sensors": ["LiDAR", "camera", "ultrasonic", "proximity sensor"],
        "manipulators": ["gripper", "drill", "welding tool", "cable hoist"],
        "communication_protocols": ["Wi-Fi", "Bluetooth"],
        "special_functions": ["precise alignment", "balance control"],
        "safety_features": ["fall detection", "collision avoidance", "emergency stop"],
        "sensor_range": 12.0,
        "processing_power": 7.0,
        "autonomy_level": "autonomous",
        "payload_capacity": 20.0,  
        "reach": 10.0,  
        "battery_life": 65.0,
        "size": (1.5, 1.0, 2.0),
        "adaptability": True
    }
]

def generate_random_robot_profile_strict(robot_id: str, grid: List[List[int]], occupied_locations: set, choice = -1) -> CapabilityProfile:
    if choice == -1:
        p = random.choice(STRICT_ROBOT_PROFILES)
    else: 
        p = STRICT_ROBOT_PROFILES[choice]

    return CapabilityProfile(
        robot_id=robot_id,
        mobility_type=p["mobility_type"],
        max_speed=1.0,
        payload_capacity=float(p["payload_capacity"]),
        reach=float(p["reach"]),
        battery_life=float(p["battery_life"]),
        size=tuple(p["size"]),
        environmental_resistance=list(p["environmental_resistance"]),
        sensors=list(p["sensors"]),
        sensor_range=float(p["sensor_range"]),
        manipulators=list(p["manipulators"]),
        communication_protocols=list(p["communication_protocols"]),
        processing_power=float(p["processing_power"]),
        autonomy_level=_normalize_autonomy(p["autonomy_level"]),
        special_functions=list(p["special_functions"]),
        safety_features=list(p["safety_features"]),
        adaptability=bool(p["adaptability"]),
        location=get_random_free_position(grid, occupied_locations),
        current_task=None,
        remaining_distance=0.0,
        time_on_task=0,
        tasks_attempted=0,
        tasks_successful=0,
        current_path=[],
        assigned=False
    )

#NAV_CONSTRAINTS = ["elevator", "stairs", "shelves", "no loud noises allowed", "narrow spaces", "low ceilings", "uneven floors",
#                   "low visibility", "slippery", "crowded", "loose debris", "no-fly zone", "windy", "dense obstructions", "smooth surfaces"]
#ENV_CONDITIONS = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant", "wind-resistant"]

STRICT_TASK_PROFILES = [
    {
        "task_type": "utilities", #gripper
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(5, 8),
        "difficulty": random.randint(5, 8),
        "navigation_constraints": _sample(["uneven floors", "loose debris", "slippery"], 0, 3),
        "required_capabilities": _sample(["payload >= 10", "reach >= 3"], 0, 2),
        "environmental_conditions": _sample(["weatherproof", "dustproof", "shock-resistance"], 0, 3),
        "tools_needed": [["LiDAR", "camera", "proximity sensor"], ["gripper"]],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "balance control", "emergency stop"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    },
    {
        "task_type": "utilities", #bucket
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(5, 8),
        "difficulty": random.randint(5, 8),
        "navigation_constraints": _sample(["uneven floors", "loose debris", "slippery"], 0, 3),
        "required_capabilities": _sample(["payload >= 10", "reach >= 3"], 0, 2),
        "environmental_conditions": _sample(["weatherproof", "dustproof", "shock-resistance"], 0, 3),
        "tools_needed": [["LiDAR", "camera", "proximity sensor"], ["hydraulic bucket"]],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "balance control", "emergency stop"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    },
    {
        "task_type": "debris", #gripper
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(4, 7),
        "difficulty": random.randint(4, 7),
        "navigation_constraints": _sample(["loose debris", "uneven floors", "crowded"], 0, 2),
        "required_capabilities": [f"payload >= {random.randint(10,15)}", f"reach >= {random.randint(0,3)}"],
        "environmental_conditions": _sample(["weatherproof", "wind-resistant"], 0, 2),
        "tools_needed": [["LiDAR", "camera", "ultrasonic", "proximity sensor"], _sample(["gripper"], 1, 1)],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "balance control", "emergency stop"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    },
    {
        "task_type": "debris", #bucket
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(4, 7),
        "difficulty": random.randint(4, 7),
        "navigation_constraints": _sample(["loose debris", "uneven floors", "crowded"], 0, 2),
        "required_capabilities": [f"payload >= {random.randint(10,15)}", f"reach >= {random.randint(0,3)}"],
        "environmental_conditions": _sample(["weatherproof", "wind-resistant"], 0, 2),
        "tools_needed": [["LiDAR", "camera", "ultrasonic", "proximity sensor"], ["hydraulic bucket"]],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "balance control", "emergency stop"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    },
    {
        "task_type": "delivery",
        "priority_level": random.choice(["low", "medium"]),
        "reward": random.randint(2, 5),
        "difficulty": random.randint(2, 5),
        "navigation_constraints": _sample(["uneven floors", "crowded", "slippery", "elevator", "stairs"], 0),
        "required_capabilities": _sample(["payload capacity >= 1.0", "payload capacity >= 5.0", "payload capacity >= 10.0"], 1, 1),
        "environmental_conditions": _sample(["weatherproof", "dustproof"], 1, 1),
        "tools_needed": [["camera", "proximity sensor", "GPS"], ["gripper"]],
        "communication_requirements": ["Wi-Fi", "4G"],
        "safety_protocols": ["obstacle detection", "emergency stop"],
        "duration": random.randint(5, 10),
        "performance_metric": "time taken",
    },
    {
        "task_type": "assembly",
        "priority_level": random.choice(["medium", "high"]),
        "reward": random.randint(5, 7),
        "difficulty": random.randint(5, 7),
        "navigation_constraints": _sample(["crowded", "loose debris", "slippery", "low ceilings"], 0, 2),
        "required_capabilities": _sample(["reach >= 1.5", "reach >= 3.0", "reach >= 5.0", "payload capacity >= 10.0"], 1, 1),
        "environmental_conditions": _sample(["dustproof", "shock-resistant", "heat-resistant"], 0, 3),
        "tools_needed": [["camera", "infrared", "dispenser"], ["gripper", "drill", "welding tool"]],
        "communication_requirements": ["Wi-Fi", "Radio"],
        "safety_protocols": ["collision avoidance", "emergency stop"],
        "duration": random.randint(8, 15),
        "performance_metric": "accuracy",
    },
    {
        "task_type": "excavate",
        "priority_level": random.choice(["high", "urgent"]),
        "reward": random.randint(6, 9),
        "difficulty": random.randint(6, 8),
        "navigation_constraints": _sample(["uneven floors", "loose debris", "low visibility"], 0, 3),
        "required_capabilities": [f"payload >= {random.randint(15, 20)}", f"reach >= {random.randint(2,7)}"],
        "environmental_conditions": _sample(["dustproof", "shock-resistant", "weatherproof"], 0, 3),
        "tools_needed": [["LiDAR", "camera", "proximity sensor"], ["hydraulic bucket"]],
        "communication_requirements": ["Radio"],
        "safety_protocols": ["overload protection", "obstacle detection"],
        "duration": random.randint(10, 18),
        "performance_metric": "safety compliance"
    },
    {
        "task_type": "item elevation",
        "priority_level": random.choice(["low", "medium"]),
        "reward": random.randint(4, 8),
        "difficulty": random.randint(4, 8),
        "navigation_constraints": _sample(["low visibility", "crowded", "uneven floors"], 0, 3),
        "required_capabilities": [f"reach >= {random.randint(15, 25)}", "payload >= 15"],
        "environmental_conditions": _sample(["weatherproof", "wind-resistant"], 0, 2),
        "tools_needed": [["camera", "GPS", "proximity sensor", "ultrasonic"], ["cable hoist", "gripper"]],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "emergency stop"],
        "duration": random.randint(3, 8),
        "performance_metric": "safety compliance"
    },
    {
        "task_type": "lay bricks",
        "priority_level": "medium",
        "reward": random.randint(6, 12),
        "difficulty": random.randint(6, 12),
        "navigation_constraints": _sample(["crowded", "low visibility", "windy"], 0, 3),
        "required_capabilities": [f"reach >= {random.randint(2,4)}", f"payload >= {random.randint(8,10)}"],
        "environmental_conditions": _sample(["dustproof", "weatherproof"], 0, 2),
        "tools_needed": [["LiDAR", "camera", "proximity sensor"], ["gripper", "dispenser"]],
        "communication_requirements": ["Wi-Fi"],
        "safety_protocols": ["collision avoidance", "overheat protection"],
        "duration": random.randint(8, 20),
        "performance_metric": "accuracy"
    },
    {
        "task_type": "scaffold",
        "priority_level": "medium",
        "reward": random.randint(5, 8),
        "difficulty": random.randint(5, 8),
        "navigation_constraints": _sample(["narrow spaces", "low ceilings", "windy"], 0, 3),
        "required_capabilities": [f"payload >= {random.randint(8, 10)}", f"reach >= {random.randint(2, 3)}"],
        "environmental_conditions": _sample(["weatherproof", "wind-resistant"], 0, 2),
        "tools_needed": [["LiDAR", "camera", "ultrasonic", "proximity sensor"], ["gripper", "drill"]],
        "communication_requirements": ["Radio", "Wi-Fi"],
        "safety_protocols": ["overload protection", "balance control", "emergency stop"],
        "duration": random.randint(10, 15),
        "performance_metric": "safety compliance"
    }
]

def generate_random_task_description_strict(task_id: str, grid: List[List[int]], occupied_locations: set, tasks, choice = -1) -> TaskDescription:
    PROFILE_POOL = _sample(STRICT_TASK_PROFILES[0:2], 1, 1) + _sample(STRICT_TASK_PROFILES[2:4], 1, 1) + STRICT_TASK_PROFILES[4:]

    if choice == -1:
        p = random.choice(PROFILE_POOL)
    else: 
        p = STRICT_TASK_PROFILES[choice]

    return TaskDescription(
        task_id=task_id,
        task_type=p["task_type"],
        objective=f"Perform {p['task_type']} task",
        priority_level=p["priority_level"],
        reward=float(p["reward"]),
        difficulty=float(p["difficulty"]),
        location=get_unique_task_location(tasks, grid, occupied_locations),
        navigation_constraints=list(p.get("navigation_constraints", [])),
        required_capabilities=list(p.get("required_capabilities", [])),  # list of strings
        time_window=(f"{random.randint(8, 17)}:00", f"{random.randint(18, 23)}:00"),
        environmental_conditions=list(p.get("environmental_conditions", [])),
        dependencies=[],
        tools_needed=list(p.get("tools_needed", [])),
        communication_requirements=[c for c in p.get("communication_requirements", []) if c in COMM_PROTOCOLS],
        safety_protocols=list(p.get("safety_protocols", [])),
        performance_metrics=p.get("performance_metric", "completion rate"),
        success_criteria="Task completed within time window",
        assigned_robot=None,
        time_to_complete=float(p.get("duration", 10.0)),
        assigned=False
    )


In [7]:
from scipy.optimize import linear_sum_assignment

def jv_task_allocation(matrix):
    matrix = np.array(matrix)
    max_val = np.max(matrix)
    
    cost_matrix = max_val - matrix
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    assignment = [(int(r), int(c)) for r, c in zip(row_ind, col_ind) if matrix[r, c] > 0]
    
    return assignment


In [ ]:
import csv

map_file = r"test_small_open.map"
grid = load_map(map_file) 

robot_sizes = [10, 50, 
               #100, 500, 1000
               ]

num_repititions = 5
if __name__ == "__main__":
    with open('VotingData.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        for num_robots in robot_sizes: 
            task_sizes = [
                int(num_robots * 0.5),
                int(num_robots * 0.75),
                num_robots,  
                int(num_robots * 1.25),
                int(num_robots * 1.5)
            ]
            for num_tasks in task_sizes:
                for num_candidates in [10, 25, 
                                       50,
                                       100
                                    ]:      
                    for rep in range(num_repititions):     
                        robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(num_robots)]
                        tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(num_tasks)]
                        suitability_matrix = calculate_suitability_matrix(robots, tasks, "evaluate_suitability_new")
                            
                        voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]

                        for method in voting_methods:
                            output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, num_candidates, method)
                            writer.writerow([method, num_robots, num_tasks, num_candidates, score, length])
                                
                        allocation_methods = [cbba_task_allocation, ssia_task_allocation, ilp_task_allocation, jv_task_allocation]
                        allocation_names = ["cbba_task_allocation", "ssia_task_allocation", "ilp_task_allocation", "jv_task_allocation"]
                        for method_idx in range(len(allocation_methods)):
                            output, score, length = assign_tasks_with_method(allocation_methods[method_idx], suitability_matrix)
                            writer.writerow([allocation_names[method_idx], num_robots, num_tasks, num_candidates, score, length])

In [47]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.anova import anova_lm

# Load the data
file_path = 'VotingData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define categorical and continuous variables
categorical_factors = ['Method']  # Replace with actual categorical column names
continuous_factors = ['Robots', 'Tasks', 'Candidates']   # Replace with actual continuous column names
# dependent_var = 'Score_Voting'                   # Replace with your dependent variable name
dependent_var = 'Score'                   # Replace with your dependent variable name

# Convert categorical variables to category dtype
for factor in categorical_factors:
    data[factor] = data[factor].astype('category')

# Define the formula for the ANCOVA model
formula = f"{dependent_var} ~ {' * '.join(categorical_factors)} + {' + '.join(continuous_factors)}"

# Fit the ANCOVA model
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA/ANCOVA

# Print ANCOVA results
print("Two-way ANCOVA results:")
print(anova_table)

# Post hoc analysis: Pairwise comparisons on adjusted means
print("\nPost hoc analysis (pairwise comparisons):")
for factor in categorical_factors:
    print(f"\nComparisons within {factor} (adjusted for covariates):")
    
    # Pairwise comparisons using t-tests
    mc = MultiComparison(data[dependent_var], data[factor])
    tukey_result = mc.tukeyhsd()
    
    # Display Tukey's test results for adjusted means
    print(tukey_result)



# Define categorical and continuous variables
categorical_factors = ['Method']  # Replace with actual categorical column names
continuous_factors = ['Robots', 'Tasks', 'Candidates']   # Replace with actual continuous column names
# dependent_var = 'Score_Voting'                   # Replace with your dependent variable name
dependent_var = 'Length'                   # Replace with your dependent variable name

# Convert categorical variables to category dtype
for factor in categorical_factors:
    data[factor] = data[factor].astype('category')

# Define the formula for the ANCOVA model
formula = f"{dependent_var} ~ {' * '.join(categorical_factors)} + {' + '.join(continuous_factors)}"

# Fit the ANCOVA model
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA/ANCOVA

# Print ANCOVA results
print("Two-way ANCOVA results:")
print(anova_table)

# Post hoc analysis: Pairwise comparisons on adjusted means
print("\nPost hoc analysis (pairwise comparisons):")
for factor in categorical_factors:
    print(f"\nComparisons within {factor} (adjusted for covariates):")
    
    # Pairwise comparisons using t-tests
    mc = MultiComparison(data[dependent_var], data[factor])
    tukey_result = mc.tukeyhsd()
    
    # Display Tukey's test results for adjusted means
    print(tukey_result)

Two-way ANCOVA results:
                  sum_sq      df            F         PR(>F)
Method      7.984341e+09     9.0   359.255049   0.000000e+00
Robots      1.113033e+09     1.0   450.728019   7.794112e-96
Tasks       3.035347e+09     1.0  1229.177848  6.552230e-241
Candidates  4.460236e+05     1.0     0.180619   6.708599e-01
Residual    1.231496e+10  4987.0          NaN            NaN

Post hoc analysis (pairwise comparisons):

Comparisons within Method (adjusted for covariates):
                                Multiple Comparison of Means - Tukey HSD, FWER=0.05                                 
              group1                             group2                meandiff  p-adj    lower      upper    reject
--------------------------------------------------------------------------------------------------------------------
              cbba_task_allocation                ilp_task_allocation   352.5287 0.7945  -302.9017  1007.9592  False
              cbba_task_allocation           

In [ ]:
import psutil

def suitability_all_zero(suitability_matrix):
    return all(value == 0 for row in suitability_matrix for value in row)

def main_simulation(output: tuple[list[tuple[int, int]],list[int],list[int]], robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, grid: List[List[int]], map_dict: dict, suitability_method: str, suitability_matrix: np.ndarray, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    print(f"SUITABILITY METHOD: {suitability_method}")
    
    initial_positions = set()
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = len(tasks)
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions) # use the occcupied positions as the current positions for CBS, this is just as a occupation check, not a start position

    assigned_pairs = output[0]  
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True

    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }
    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    # print(f"ASSIGNED PAIRS: {assigned_pairs}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    print(f"AGENTS LIST: {agents}")

    # Create the input data dictionary for CBS, this will be passed to the CBS planner
    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
        # could possibly fall back on the simple method here if we get a lot of issues
    else:

        # Iterate through the agents and their schedules
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            # Iterate through the schedule (list of dictionaries)
            for point in schedule:
                # Access each entry in the schedule
                # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                # Assign the paths to the robots
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    # Keep track of previous state to not run CBS when there are no changes
    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
 
    # End the simulation if nothing changes for 3 timesteps (hopefully the tasks are done and the robots arent moving)
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        print(f"START POSITIONS: {start_positions}")
        print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        # before each time step, refresh the unassigned robots and tasks lists
        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        # Periodically add new tasks and robots
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks_strict(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots_strict(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        # Periodically remove robots
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1: # Otherwise will break CBS, we need at least one agent for things to run smoothly
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)
 

        # Reassign unassigned robots to unassigned tasks
        # TODO: change this to make sure newly assigned robot is more suitable for the task
        # so we will run it when there are unassigned robots AND a state change, otherwise theres no reason to keep checking
        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(
                robots, tasks, num_candidates, voting_method, suitability_method,
                unassigned_robots, unassigned_tasks, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score
            # regenerate the master assigned robots dictionary
            assigned_robots = {
                r.robot_id: r.current_task.task_id
                for r in robots
                if r.assigned and r.current_task is not None
            }

            print(f"ROBOTS REASSIGNED")
            # print(f"NEW ASSIGNMENTS FROM REASSIGNMENT: {new_assignments}")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")
 
 
        # Update start and goal positions before cbs
        for robot in robots:
            if robot.assigned and robot.current_task:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }
 
        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            # NOTE: use occupied positions as current positions for running CBS, update goal positions whenever needed
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            agents = build_cbs_agents(robots, start_positions, goal_positions)

            print(f"AGENTS LIST: {agents}")
 
            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # if time_steps_unchanged < 3:
            #     time_steps_unchanged += 1
            #     print("No state change, skip CBS...")
            #     print(f"ALL ROBOTS: {len(robots)}")
            #     print(f"ALL TASKS: {len(tasks)}")
            #     print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            #     print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # else:
            #     break

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

def benchmark_simulation(output: tuple[list[tuple[int, int]],list[int],list[int]], robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, grid: List[List[int]], map_dict: dict, suitability_method: str, suitability_matrix: np.ndarray, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    start_time = time.time()
    main_simulation(output, robots, tasks, num_candidates, voting_method, grid, map_dict, suitability_method, suitability_matrix, max_time_steps, add_tasks, add_robots, remove_robots, tasks_to_add, robots_to_add, robots_to_remove)
    end_time = time.time()
    execution_time = end_time - start_time

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used

    print(f"Simulation completed in {execution_time:.2f} seconds.")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage / (1024 * 1024)} MB")

if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    # suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    suitability_methods = ["evaluate_suitability_new"]
    # suitability_methods = ["evaluate_suitability_distance"]
    max_time_steps = 100
    add_tasks = True
    add_robots = False
    remove_robots = False

    map_file = r"test_small_open.map"
    grid = load_map(map_file) # 2D list of 0/1 representing the map
    dims = (len(grid), len(grid[0])) # dimensions of the map grid
    obstacles = create_obstacle_list(grid) # list of obstacle coordinates
    # Create the map dictionary to pass to the CBS planner
    map_dict = {
        'dimension': dims,
        'obstacles': obstacles
    }

    # print(f"MAP DICTIONARY: {map_dict}")
    for i in [5]:
        for j in [5]:
            for nc in [5]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # get initial positions list to reuse later
                            # generate_random_robot_profile now takes in the map grid and occupied positions and passes them to get_random_free_position function
                            robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(i)]
                            # changed how tasks are made here so there are no overlapping tasks
                            tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(j)]

                            suitability_matrix = calculate_suitability_matrix(robots, tasks, sm)

                            while suitability_all_zero(suitability_matrix):
                                robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(i)]
                                tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(j)]
                                suitability_matrix = calculate_suitability_matrix(robots, tasks, sm)

                            output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, nc, vm)
                            cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
                            ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
                            ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
                            outputs = [output, cbba_output, ssia_output, ilp_output]
                                # param_combinations = []
                                # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
                                # with multiprocessing.Pool() as pool:
                                # pool.starmap(main_simulation, param_combinations)
                                # main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)
                            for o in outputs:
                                benchmark_simulation(o, copy.deepcopy(robots), copy.deepcopy(tasks), nc, vm, grid, map_dict, sm, suitability_matrix, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)

In [8]:
def navigation_suitability(robot_mobility_type: str, robot_size: Tuple[float, float, float], task_constraints: List[str]) -> float:
    """
    Evaluates the suitability of a robot for navigating a task environment based on mobility type, size, and navigation constraints.
    
    Parameters:
        robot_mobility_type: The mobility type of the robot (e.g., "wheeled", "tracked", "legged", "aerial", "hovering", "climbing").
        robot_size: A tuple representing the robot's dimensions (length, width, height).
        task_constraints: A list of navigation constraints for the task environment.
    
    Returns:
        A float score representing the suitability for navigation. Returns 0 if there is a critical mismatch that prevents navigation.
    """

    # Initialize the score
    score = 0.0

    # Define size thresholds for narrow spaces, low ceilings, etc.
    narrow_space_threshold = 2.0  # Width limit for narrow spaces
    low_ceiling_threshold = 2.0   # Height limit for low ceilings

    # Handle each task constraint based on mobility type
    for constraint in task_constraints:
        # Constraint: Elevator access
        if constraint == "elevator":
            score += 1.0

        # Constraint: Stairs
        elif constraint == "stairs":
            if robot_mobility_type in ["legged", "aerial", "hovering", "climbing"]:
                score += 1.0
            else:
                return 0.0

        # Constraint: Shelves
        elif constraint == "shelves":
            if robot_size[2] < low_ceiling_threshold or robot_mobility_type in ["aerial", "climbing", "hovering"]:
                score += 1.0  # Only smaller robots can access shelves effectively

        # Constraint: No loud noises allowed
        elif constraint == "no loud noises allowed":
            if robot_mobility_type in ["legged", "hovering"]:
                score += 1.0  # Quieter mobility types

        # Constraint: Narrow spaces
        elif constraint == "narrow spaces":
            if robot_size[1] <= narrow_space_threshold:
                score += 1.0
            else:
                return 0.0  # Larger robots cannot pass through narrow spaces

        # Constraint: Low ceilings
        elif constraint == "low ceilings":
            if robot_size[2] <= low_ceiling_threshold:
                score += 1.0
            else:
                return 0.0  # Tall robots cannot navigate in areas with low ceilings

        # Constraint: Uneven floors
        elif constraint == "uneven floors":
            if robot_mobility_type in ["tracked", "legged", "hovering", "aerial"]:
                score += 1.0  # These types handle uneven floors well

        # Constraint: Low visibility
        elif constraint == "low visibility":
            if robot_mobility_type in ["wheeled", "tracked", "legged"]:
                score += 1.0  # Infrared or LiDAR-equipped robots are suitable

        # Constraint: Slippery surfaces
        elif constraint == "slippery":
            if robot_mobility_type in ["tracked", "hovering", "aerial"]:
                score += 1.0  # Hovering and tracked types handle slippery surfaces better
            elif robot_mobility_type in ["wheeled", "legged"]:
                return 0.0  # Wheeled and legged robots are unsuitable on slippery floors

        # Constraint: Crowded environments
        elif constraint == "crowded":
            if robot_size[0] <= 1.0 and robot_size[1] <= 1.0:
                score += 1.0  # Smaller robots are more suitable in crowded environments
            else:
                score += 0.5  # Larger robots get a lower score

        # Constraint: Loose debris
        elif constraint == "loose debris":
            if robot_mobility_type in ["aerial", "hovering", "tracked"]:
                score += 1.0  # Aerial and hovering robots handle debris better
            elif robot_mobility_type == "legged":
                return 0.0  # Legged robots are unsuitable
            else:
                score += 0.5

        # Constraint: No-fly zone
        elif constraint == "no-fly zone":
            if robot_mobility_type == "aerial":
                return 0.0  # Aerial robots cannot navigate in no-fly zones
            else:
                score += 0.5  # Other mobility types are unaffected

        # Constraint: Windy conditions
        elif constraint == "windy":
            if robot_mobility_type == "aerial":
                return 0.0  # Aerial robots struggle in windy conditions
            else:
                score += 0.5  # All other types are more stable in wind

        # Constraint: Dense obstructions (e.g., tree branches, hanging cables)
        elif constraint == "dense obstructions":
            if robot_mobility_type in ["aerial", "legged"]:
                return 0.0  # Aerial and legged robots are unsuitable in dense obstruction areas
            else:
                score += 0.5  # Other types may navigate dense areas on the ground

        # Constraint: Smooth floors
        elif constraint == "smooth surfaces":
            if robot_mobility_type == "climbing":
                return 0.0  # Climbing robots are less suited for smooth surfaces

    # Final suitability score (0 if any constraint returns 0)
    return score if score > 0 else 0.0


def evaluate_suitability_loose(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    total_weight = 0.0  # for normalization

    weights = {
        "payload": 3.0,
        "manipulators": 4.0,
        "sensors": 3.0,
        "communication": 0.5,
        "safety": 1.0,
        "environmental": 1.0,
        "reach": 2.0,
        "sensor_range": 1.0,
        "proximity": 1.0,
        "autonomy_match": 0.5,
        "battery_duration": 2.0,
        "special_functions": 2.0,
        "processing_power": 1.0,
        "adaptability": 0.5,
        "navigation": 2.0,
    }

    # Payload
    total_weight += weights["payload"]
    if any("payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1]) for req in task.required_capabilities):
        score += 0.0
    else:
        score += weights["payload"]

    # Reach
    total_weight += weights["reach"]
    if any("reach" in req and robot.reach < float(req.split(">= ")[-1]) for req in task.required_capabilities):
        score += 0.0
    else:
        score += weights["reach"]

    # Manipulators (tools_needed[1] is manipulators list)
    total_weight += weights["manipulators"]
    if task.tools_needed:
        if ("cable hoist" in task.tools_needed[1] and "cable hoist" not in robot.manipulators and robot.mobility_type in ["hovering", "aerial"]):
            task.tools_needed[1].remove("cable hoist")
        matched_tools = sum(tool in robot.manipulators for tool in task.tools_needed[1])
        tool_score = matched_tools / len(task.tools_needed[1])
        score += weights["manipulators"] * tool_score

    # Sensors (tools_needed[0] is sensors list)
    total_weight += weights["sensors"]
    if task.tools_needed:
        matched_tools = sum(tool in robot.sensors for tool in task.tools_needed[0])
        tool_score = matched_tools / len(task.tools_needed[0])
        score += weights["sensors"] * tool_score

    # Communication
    total_weight += weights["communication"]
    if task.communication_requirements:
        matched_comm = sum(proto in robot.communication_protocols for proto in task.communication_requirements)
        comm_score = matched_comm / len(task.communication_requirements)
        score += weights["communication"] * comm_score

    # Safety
    total_weight += weights["safety"]
    if robot.safety_features and task.safety_protocols:
        matched_safety = sum(safety in robot.safety_features for safety in task.safety_protocols)
        safety_score = matched_safety / len(task.safety_protocols)
        score += weights["safety"] * safety_score

    # Environmental
    total_weight += weights["environmental"]
    if robot.environmental_resistance and task.environmental_conditions:
        matched_environmental = sum(condition in robot.environmental_resistance for condition in task.environmental_conditions)
        environmental_score = matched_environmental / len(task.environmental_conditions)
        score += weights["environmental"] * environmental_score

    # Navigation
    total_weight += weights["navigation"]
    if task.navigation_constraints:
        navigation_score = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        score += weights["navigation"] * navigation_score

    # Sensor range
    total_weight += weights["sensor_range"]
    distance_to_task = len(robot.current_path) - 1
    sensor_score = 1.0 if robot.sensor_range >= distance_to_task else \
                   0.5 if robot.sensor_range >= distance_to_task / 2 else 0.0
    score += weights["sensor_range"] * sensor_score

    # Proximity
    total_weight += weights["proximity"]
    if distance_to_task < 20.0:
        score += weights["proximity"]
    elif distance_to_task < 50.0:
        score += weights["proximity"] * 0.5

    # Autonomy
    total_weight += weights["autonomy_match"]
    autonomy_score = 0.0
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        autonomy_score = 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        autonomy_score = 0.5
    score += weights["autonomy_match"] * autonomy_score

    # Battery
    total_weight += weights["battery_duration"]
    if ((distance_to_task / robot.max_speed) + task.time_to_complete) > robot.battery_life:
        return 0.0
    battery_score = (1.0 if robot.battery_life >= 2 * ((distance_to_task / robot.max_speed) + task.time_to_complete)
                    else 0 if ((distance_to_task / robot.max_speed) + task.time_to_complete) > robot.battery_life
                    else 0.5)
    score += weights["battery_duration"] * battery_score

    # Special functions
    total_weight += weights["special_functions"]
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "assembly": ["object recognition", "object tracking", "precise alignment"],
        "utilities": ["percise alignment", "balance control"],
        "excavate": [ "terrain leveling", "object recognition", "precise alignment"],
        "debris": ["balance control", "object recognition"],
        "level": ["terrain leveling", "object recognition"],
        "item elevation": ["precise alignment", "object tracking", "balance control"],
        "lay bricks": ["object recognition", "precise alignment"],
        "scaffold": ["precise alignment", "balance control"],
        "remove scaffold": ["object recognition", "object tracking", "precise alignment"],
    }
    required_functions = task_function_mapping[task.task_type]
    matched_special_functions = sum(special in robot.special_functions for special in required_functions)
    special_functions_score = matched_special_functions / len(required_functions)
    score += weights["special_functions"] * special_functions_score

    # Processing power
    total_weight += weights["processing_power"]
    proc_score = 0.0
    if task.difficulty > 7:
        if robot.processing_power >= 5.0:
            proc_score = 1.0
        elif robot.processing_power >= 3.0:
            proc_score = 0.75
        else:
            proc_score = 0.5
    elif task.difficulty > 4:
        if robot.processing_power >= 3.0:
            proc_score = 1.0
        else:
            proc_score = 0.75
    elif task.difficulty > 2:
        if robot.processing_power >= 1.5:
            proc_score = 1.0
        else:
            proc_score = 0.75
    score += weights["processing_power"] * proc_score

    # Adaptability
    total_weight += weights["adaptability"]
    score += weights["adaptability"] * (1.0 if robot.adaptability else 0.0)

    # Reward/Difficulty ratio
    reward_score = (task.reward / max(task.difficulty, 1.0))
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    score += priority_multiplier * (reward_score / (reward_score + 10.0))  # squash into (0,1)

    # Normalize
    if total_weight > 0:
        final_score = score / total_weight
    else:
        final_score = 0.0

    return max(0.0, min(1.0, final_score))

def evaluate_suitability_strict(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    total_weight = 0.0  # for normalization

    weights = {
        "payload": 3.0,
        "manipulators": 4.0,
        "sensors": 3.0,
        "communication": 0.5,
        "safety": 1.0,
        "environmental": 1.0,
        "reach": 2.0,
        "sensor_range": 1.0,
        "proximity": 1.0,
        "autonomy_match": 0.5,
        "battery_duration": 2.0,
        "special_functions": 2.0,
        "processing_power": 1.0,
        "adaptability": 0.5,
        "navigation": 2.0,
    }

    # Payload
    total_weight += weights["payload"]
    if any("payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1]) for req in task.required_capabilities):
        return 0.0
    else:
        score += weights["payload"]

    # Reach
    total_weight += weights["reach"]
    if any("reach" in req and robot.reach < float(req.split(">= ")[-1]) for req in task.required_capabilities):
        return 0.0
    else:
        score += weights["reach"]

    # Manipulators (tools_needed[1] is manipulators list)
    total_weight += weights["manipulators"]
    if task.tools_needed:
        if ("cable hoist" in task.tools_needed[1] and "cable hoist" not in robot.manipulators and robot.mobility_type in ["hovering", "aerial"]):
            task.tools_needed[1].remove("cable hoist")
        matched_tools = sum(tool in robot.manipulators for tool in task.tools_needed[1])
        tool_score = matched_tools / len(task.tools_needed[1])
        if tool_score != 1:
            return 0.0
        score += weights["manipulators"] * tool_score

    # Sensors (tools_needed[0] is sensors list)
    total_weight += weights["sensors"]
    if task.tools_needed:
        matched_tools = sum(tool in robot.sensors for tool in task.tools_needed[0])
        tool_score = matched_tools / len(task.tools_needed[0])
        if tool_score != 1:
            return 0.0
        score += weights["sensors"] * tool_score

    # Communication
    total_weight += weights["communication"]
    if task.communication_requirements:
        matched_comm = sum(proto in robot.communication_protocols for proto in task.communication_requirements)
        comm_score = matched_comm / len(task.communication_requirements)
        if comm_score != 1:
            return 0.0
        score += weights["communication"] * comm_score

    # Safety
    total_weight += weights["safety"]
    if robot.safety_features and task.safety_protocols:
        matched_safety = sum(safety in robot.safety_features for safety in task.safety_protocols)
        safety_score = matched_safety / len(task.safety_protocols)
        if safety_score != 1:
            return 0.0
        score += weights["safety"] * safety_score

    # Environmental
    total_weight += weights["environmental"]
    if robot.environmental_resistance and task.environmental_conditions:
        matched_environmental = sum(condition in robot.environmental_resistance for condition in task.environmental_conditions)
        environmental_score = matched_environmental / len(task.environmental_conditions)
        if environmental_score != 1:
            return 0.0
        score += weights["environmental"] * environmental_score

    # Navigation
    total_weight += weights["navigation"]
    if task.navigation_constraints:
        navigation_score = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_score == 0:
            return 0.0
        score += weights["navigation"] * navigation_score

    # Sensor range
    total_weight += weights["sensor_range"]
    distance_to_task = len(robot.current_path) - 1
    sensor_score = 1.0 if robot.sensor_range >= distance_to_task else \
                   0.5 if robot.sensor_range >= distance_to_task / 2 else 0.0
    score += weights["sensor_range"] * sensor_score

    # Proximity
    total_weight += weights["proximity"]
    if distance_to_task < 20.0:
        score += weights["proximity"]
    elif distance_to_task < 50.0:
        score += weights["proximity"] * 0.5

    # Autonomy
    total_weight += weights["autonomy_match"]
    autonomy_score = 0.0
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        autonomy_score = 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        autonomy_score = 0.5
    score += weights["autonomy_match"] * autonomy_score

    # Battery
    total_weight += weights["battery_duration"]
    if ((distance_to_task / robot.max_speed) + task.time_to_complete) > robot.battery_life:
        return 0.0
    battery_score = 1.0 if robot.battery_life >= 2 * ((distance_to_task / robot.max_speed) + task.time_to_complete) else 0.5
    score += weights["battery_duration"] * battery_score

    # Special functions
    total_weight += weights["special_functions"]
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "assembly": ["object recognition", "object tracking", "precise alignment"],
        "utilities": ["percise alignment", "balance control"],
        "excavate": [ "terrain leveling", "object recognition", "precise alignment"],
        "debris": ["balance control", "object recognition"],
        "level": ["terrain leveling", "object recognition"],
        "item elevation": ["precise alignment", "object tracking", "balance control"],
        "lay bricks": ["object recognition", "precise alignment"],
        "scaffold": ["precise alignment", "balance control"],
        "remove scaffold": ["object recognition", "object tracking", "precise alignment"],
    }
    required_functions = task_function_mapping[task.task_type]
    matched_special_functions = sum(special in robot.special_functions for special in required_functions)
    special_functions_score = matched_special_functions / len(required_functions)
    score += weights["special_functions"] * special_functions_score

    # Processing power
    total_weight += weights["processing_power"]
    proc_score = 0.0
    if task.difficulty > 7:
        if robot.processing_power >= 5.0:
            proc_score = 1.0
        elif robot.processing_power >= 3.0:
            proc_score = 0.75
        else:
            proc_score = 0.5
    elif task.difficulty > 4:
        if robot.processing_power >= 3.0:
            proc_score = 1.0
        else:
            proc_score = 0.75
    elif task.difficulty > 2:
        if robot.processing_power >= 1.5:
            proc_score = 1.0
        else:
            proc_score = 0.75
    score += weights["processing_power"] * proc_score

    # Adaptability
    total_weight += weights["adaptability"]
    score += weights["adaptability"] * (1.0 if robot.adaptability else 0.0)

    # Reward/Difficulty ratio
    reward_score = (task.reward / max(task.difficulty, 1.0))
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    score += priority_multiplier * (reward_score / (reward_score + 10.0))  # squash into (0,1)

    # Normalize
    if total_weight > 0:
        final_score = score / total_weight
    else:
        final_score = 0.0

    return max(0.0, min(1.0, final_score))

def evaluate_suitability_distance(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if robot.sensors:
        if task.tools_needed and not all(item in robot.sensors for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools

    if robot.manipulators:
        if task.tools_needed and not all(item in robot.manipulators for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if robot.safety_features and task.safety_protocols:
        if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
            score += 0.0  # Suitability is zero if the robot lacks required safety features
        else:
            score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if robot.environmental_resistance and task.environmental_conditions:
        if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
            score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
        else:
            score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            score += 0.0
        else:
            score += navigation_match

    # NOTE: CHANGED TO WORK WITH COORDINATES
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    # stop suitability matrix from going negative
    distance_to_task = max(0, len(robot.current_path) - 1)
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += 1.0
        elif robot.sensor_range >= distance_to_task/2:
            score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
#     if distance_to_task < 20.0:
#         score += 1.0
#     elif distance_to_task < 50.0:
#         score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    if robot.special_functions:
        for function in robot.special_functions:
            if function in required_functions:
                score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    #if task.task_type in robot.preferred_tasks:
    #    score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Weight score by distance to task
    # NOTE: IF THE ROBOT IS AT THE TASK THIS CAN CAUSE A DIVIDE BY ZERO ERROR
    score = score / (distance_to_task + 1E-5)
    
    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_priority(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.
    
    Returns:
        score: A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if robot.sensors:
        if task.tools_needed and not all(item in robot.sensors for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
    if robot.manipulators:
        if task.tools_needed and not all(item in robot.manipulators for item in task.tools_needed):
            score += 0.0  # Suitability is zero if the robot lacks necessary tools
        else:
            score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, r
# obot.safety_features)
    # Check if the robot can safely perform the task
    if robot.safety_features and task.safety_protocols:
        if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
            score += 0.0  # Suitability is zero if the robot lacks required safety features
        else:
            score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if robot.environmental_resistance and task.environmental_conditions:
        if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
            score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
        else:
            score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    if task.navigation_constraints:
        navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_match == 0:
            score += 0.0
        else:
            score += navigation_match

    # NOTE: CHANGED TO WORK WITH COORDINATES
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range:
        if robot.sensor_range >= distance_to_task:
            score += 1.0
        elif robot.sensor_range >= distance_to_task/2:
            score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.time_to_complete) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.time_to_complete):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    if robot.special_functions:
        for function in robot.special_functions:
            if function in required_functions:
                score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    #if task.task_type in robot.preferred_tasks:
    #    score += 1.0

    # Score based on reward and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += reward_to_difficulty_ratio

    # Weight based on priority
    score = score * priority_multiplier
    
    # Return the final suitability score
#     print(score)
    return score


In [10]:
def evaluate_suitability_new(robot: CapabilityProfile, task: TaskDescription) -> float:
    score = 0.0
    total_weight = 0.0  # for normalization

    weights = {
        "payload": 3.0,
        "manipulators": 4.0,
        "sensors": 3.0,
        "communication": 0.5,
        "safety": 1.0,
        "environmental": 1.0,
        "reach": 2.0,
        "sensor_range": 1.0,
        "proximity": 1.0,
        "autonomy_match": 0.5,
        "battery_duration": 2.0,
        "special_functions": 2.0,
        "processing_power": 1.0,
        "adaptability": 0.5,
        "navigation": 2.0,
    }

    # Payload
    total_weight += weights["payload"]
    if any("payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1]) for req in task.required_capabilities):
        return 0.0
    else:
        score += weights["payload"]

    # Reach 
    total_weight += weights["reach"]
    if any("reach" in req and robot.reach < float(req.split(">= ")[-1]) for req in task.required_capabilities):
        score += 0.0
    else:
        score += weights["reach"]

    # Manipulators (tools_needed[1] is manipulators list)
    total_weight += weights["manipulators"]
    if task.tools_needed:
        if ("cable hoist" in task.tools_needed[1] and "cable hoist" not in robot.manipulators and robot.mobility_type in ["hovering", "aerial"]):
            task.tools_needed[1].remove("cable hoist")
        matched_tools = sum(tool in robot.manipulators for tool in task.tools_needed[1])
        tool_score = matched_tools / len(task.tools_needed[1])
        if tool_score != 1:
            return 0.0
        score += weights["manipulators"] * tool_score

    # Sensors (tools_needed[0] is sensors list)
    total_weight += weights["sensors"]
    if task.tools_needed:
        matched_tools = sum(tool in robot.sensors for tool in task.tools_needed[0])
        tool_score = matched_tools / len(task.tools_needed[0])
        score += weights["sensors"] * tool_score

    # Communication 
    total_weight += weights["communication"]
    if task.communication_requirements:
        matched_comm = sum(proto in robot.communication_protocols for proto in task.communication_requirements)
        comm_score = matched_comm / len(task.communication_requirements)
        score += weights["communication"] * comm_score

    # Safety 
    total_weight += weights["safety"]
    if robot.safety_features and task.safety_protocols:
        matched_safety = sum(safety in robot.safety_features for safety in task.safety_protocols)
        safety_score = matched_safety / len(task.safety_protocols)
        score += weights["safety"] * safety_score

    # Environmental 
    total_weight += weights["environmental"]
    if robot.environmental_resistance and task.environmental_conditions:
        matched_environmental = sum(condition in robot.environmental_resistance for condition in task.environmental_conditions)
        environmental_score = matched_environmental / len(task.environmental_conditions)
        score += weights["environmental"] * environmental_score

    # Navigation 
    total_weight += weights["navigation"]
    if task.navigation_constraints:
        navigation_score = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
        if navigation_score == 0:
            return 0.0
        score += weights["navigation"] * navigation_score

    # Sensor range 
    total_weight += weights["sensor_range"]
    distance_to_task = len(robot.current_path) - 1
    sensor_score = 1.0 if robot.sensor_range >= distance_to_task else \
                   0.5 if robot.sensor_range >= distance_to_task / 2 else 0.0
    score += weights["sensor_range"] * sensor_score

    # Proximity 
    total_weight += weights["proximity"]
    if distance_to_task < 20.0:
        score += weights["proximity"]
    elif distance_to_task < 50.0:
        score += weights["proximity"] * 0.5

    # Autonomy 
    total_weight += weights["autonomy_match"]
    autonomy_score = 0.0
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        autonomy_score = 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        autonomy_score = 0.5
    score += weights["autonomy_match"] * autonomy_score

    # Battery 
    total_weight += weights["battery_duration"]
    if ((distance_to_task / robot.max_speed) + task.time_to_complete) > robot.battery_life:
        return 0.0
    battery_score = 1.0 if robot.battery_life >= 2 * ((distance_to_task / robot.max_speed) + task.time_to_complete) else 0.5
    score += weights["battery_duration"] * battery_score

    # Special functions 
    total_weight += weights["special_functions"]
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "assembly": ["object recognition", "object tracking", "precise alignment"],
        "utilities": ["percise alignment", "balance control"],
        "excavate": [ "terrain leveling", "object recognition", "precise alignment"],
        "debris": ["balance control", "object recognition"],
        "level": ["terrain leveling", "object recognition"],
        "item elevation": ["precise alignment", "object tracking", "balance control"],
        "lay bricks": ["object recognition", "precise alignment"],
        "scaffold": ["precise alignment", "balance control"],
        "remove scaffold": ["object recognition", "object tracking", "precise alignment"],
    }
    required_functions = task_function_mapping[task.task_type]
    matched_special_functions = sum(special in robot.special_functions for special in required_functions)
    special_functions_score = matched_special_functions / len(required_functions)
    score += weights["special_functions"] * special_functions_score

    # Processing power 
    total_weight += weights["processing_power"]
    proc_score = 0.0
    if task.difficulty > 7:
        if robot.processing_power >= 5.0:
            proc_score = 1.0
        elif robot.processing_power >= 3.0:
            proc_score = 0.75
        else:
            proc_score = 0.5
    elif task.difficulty > 4:
        if robot.processing_power >= 3.0:
            proc_score = 1.0
        else:
            proc_score = 0.75
    elif task.difficulty > 2:
        if robot.processing_power >= 1.5:
            proc_score = 1.0
        else:
            proc_score = 0.75
    score += weights["processing_power"] * proc_score

    # Adaptability 
    total_weight += weights["adaptability"]
    score += weights["adaptability"] * (1.0 if robot.adaptability else 0.0)

    # Reward/Difficulty ratio 
    reward_score = (task.reward / max(task.difficulty, 1.0))
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    score += priority_multiplier * (reward_score / (reward_score + 10.0))  # squash into (0,1)

    # Normalize 
    if total_weight > 0:
        final_score = score / total_weight
    else:
        final_score = 0.0

    return max(0.0, min(1.0, final_score))


In [12]:
from openai import OpenAI

def evaluate_suitability_with_llm(robot: CapabilityProfile, task: TaskDescription) -> str:
    """
    Uses a large language model (LLM) to evaluate the suitability of a robot for a given task.


    Parameters:
        robot: The CapabilityProfile of the robot.
        task: The TaskDescription of the task.


    Returns:
        score: A float score (0 to 1) representing the suitability of the robot for the task.
    """
    prompt = f"""
    You are evaluating the suitability of a robot for a specific task in a robotic system.

    Here are the details of the robot:
    - Mobility Type: {robot.mobility_type}
    - Maximum Speed: {robot.max_speed} units/sec
    - Payload Capacity: {robot.payload_capacity} kg
    - Reach: {robot.reach} meters
    - Battery Life: {robot.battery_life} hours
    - Size: {robot.size} (length, width, height in meters)
    - Environmental Resistance: {', '.join(robot.environmental_resistance) if robot.environmental_resistance  else "No environmental resistance"}
    - Sensors: {', '.join(robot.sensors) if robot.sensors else "No sensors"}
    - Sensor Range: {robot.sensor_range if robot.sensor_range else "0"} meters
    - Manipulators: {', '.join(robot.manipulators) if robot.manipulators else "No manipulators"}
    - Communication Protocols: {', '.join(robot.communication_protocols)}
    - Processing Power: {robot.processing_power} units
    - Autonomy Level: {robot.autonomy_level}
    - Special Functions: {', '.join(robot.special_functions) if robot.special_functions else "No special functions"}
    - Safety Features: {', '.join(robot.safety_features) if robot.safety_features else "No safety features"}
    - Adaptability: {"Yes" if robot.adaptability else "No"}
    - Location: {robot.location[0]}, {robot.location[1]}

    Here are the details of the task:
    - Task Type: {task.task_type}
    - Objective: {task.objective}
    - Priority Level: {task.priority_level}
    - Reward: {task.reward} units
    - Difficulty: {task.difficulty} (scale of 1 to 10)
    - Location: {task.location} (x, y, z coordinates)
    - Navigation Constraints: {', '.join(task.navigation_constraints) if task.navigation_constraints else 'None'}
    - Required Capabilities: {', '.join(task.required_capabilities)}
    - Duration: {task.time_to_complete} seconds
    - Environmental Conditions: {', '.join(task.environmental_conditions) if task.environmental_conditions else 'None'}
    - Manipulators Needed: {', '.join(task.tools_needed[1]) if task.tools_needed else 'None'}
    - Sensors Needed: {', '.join(task.tools_needed[0]) if task.tools_needed else 'None'}
    - Communication Requirements: {', '.join(task.communication_requirements) if task.communication_requirements else 'None'}
    - Safety Protocols: {', '.join(task.safety_protocols) if task.safety_protocols else 'None'}
    - Performance Metrics: {task.performance_metrics if task.performance_metrics else 'None'}
    - Success Criteria: {task.success_criteria}

    Based on the robot's capabilities and the task requirements, please rate the suitability of this robot for this task on a scale of 0 to 1, where 0 means the robot is completely unsuitable and 1 means it is perfectly suited.
    When evaluating whether the robot has the CRITICAL requirements, ONLY check the following three fields:

    1. Manipulators
    2. Navigation Constraints 
    3. Payload Capacity

    Ignore all other fields when deciding if the robot is missing critical requirements. The robot should only receive a score of 0 **if one or more of those THREE specific requirements are not satisfied**.
    Once the critical requirements (Tools Needed, Navigation Constraints, and Payload Capacity) are satisfied, evaluate all OTHER attributes. All other capabilities and requirements should still influence the score between 0 and 1 (if the critical requirements are met).
    When comparing multi-value attributes (such as Environmental Resistance, Sensors, Communication Protocols, Special Functions, or Safety Features), treat them as **matched** if the robot satisfies the **majority** of the required values. If it doesn't reach majority, but is still not 100% unmatched, give *partial credit*.  
    Only consider the attribute a **mismatch** if the robot is missing **more than half** of the values in that field.
    *Duration is how much time is required to finish a task, once the robot arrives at the task. Do NOT factor it in when calculating whether the robot can reach the task location in time*
    *Task priority level should be compared with robot's autonomy level. Task's difficulty should be compared with robot's processing power. If robot is adaptable, add to the score, automatically. A robot's special functions should be compared to the overall requirements of the task*.
    """
    
    #Provide ONLY a single number between 0 and 1. Do not include any words, punctuation, or explanations.
    score = 0.0
    try:
        client = OpenAI()
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt
        )
        score = response.output_text
    except Exception as e:
        print(f"Error: {response}")

    return score

In [ ]:
map_file = r"test_small_open.map"
grid = load_map(map_file) 

robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set(), i) for i in range(6)]

tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), [], i) for i in range(10)]

suitability_matrix = calculate_suitability_matrix(robots, tasks, "evaluate_suitability_new")

print(suitability_matrix)


CapabilityProfile(robot_id=R1, mobility_type=aerial, max_speed=1.0, payload_capacity=20.0, reach=0.0, battery_life=80.0, size=(2.0, 2.0, 2.0), environmental_resistance=['weatherproof', 'heat-resistant', 'cold-resistant'], sensors=['LiDAR', 'GPS', 'proximity sensor', 'camera'], sensor_range=25.0, manipulators=['gripper'], communication_protocols=['Wi-Fi', '4G'], processing_power=5.0, autonomy_level=fully autonomus, special_functions=['object recognition', 'object tracking', 'facial recognition'], safety_features=['obstacle detection', 'emergency stop'], adaptability=False, location=(21, 24), remaining_distance=0.0, time_on_task=0, tasks_attempted=0, tasks_successful=0)
TaskDescription(task_id=T1, task_type=utilities, objective=Perform utilities task, priority_level=high, location=(42, 27), reward=8.0, difficulty=6.0, navigation_constraints=[], required_capabilities=[], time_window=('13:00', '23:00'), environmental_conditions=[], dependencies=[], tools_needed=[['LiDAR', 'camera', 'proxim

In [22]:
llm_output = evaluate_suitability_with_llm(robot = robots[0], task = tasks[4])

print(f"LLM output: {llm_output}")

LLM output: Let's first check the CRITICAL requirements:

1. Manipulators:
   - Task requires: gripper
   - Robot has: gripper
   - Status: met

2. Navigation Constraints:
   - Task constraints: elevator, stairs, crowded
   - Robot mobility: aerial
   - Aerial robots typically do not have issues with elevators, stairs, or crowded pathways as they fly.
   - Status: met

3. Payload Capacity:
   - Task requires: >= 10.0 kg
   - Robot capacity: 20.0 kg
   - Status: met

Since all CRITICAL requirements are met, we proceed with partial/full matching of other attributes:

Comparing all attributes:

- Environmental Resistance (Task): dustproof
  (Robot): weatherproof, heat-resistant, cold-resistant
  - Does not explicitly list dustproof.
  - No match here, but since only one requirement and robot does not meet it, this is a mismatch.

- Sensors Required (Task): camera, proximity sensor, GPS
  (Robot): LiDAR, GPS, proximity sensor, camera
  - Has all three required sensors.
  - Status: matched


In [13]:
def generate_random_assignments(num_robots: int, num_tasks: int, num_assignments: int = 5) -> List[List[Tuple[Optional[int], Optional[int]]]]:
    """
    Generates a list of random assignments of robots to tasks.
    Each assignment is a list of (robot, task) pairs, where some robots or tasks may be unassigned.
    These assignments are not generated based on suitability scores.
    
    Parameters:
        num_robots: Number of available robots.
        num_tasks: Number of tasks to be assigned.
        num_assignments: Number of different random assignments to generate.
    
    Returns:
        assignments: A list of random assignments, where each assignment is a list of (robot, task) pairs.
    """
    assignments = []

    # Generate random assignments
    for _ in range(num_assignments):
        # Generate random indices for pairing
        available_robots = random.sample(range(num_robots), num_robots)
        available_tasks = random.sample(range(num_tasks), num_tasks)
        
        # Determine the number of pairs to create (limited by the smaller of num_robots or num_tasks)
        num_pairs = min(num_robots, num_tasks)

        # Pair robots and tasks using the precomputed random indices
        assigned_pairs = [(available_robots[i], available_tasks[i]) for i in range(num_pairs)]

        # Determine unassigned robots and tasks
        unassigned_robots = available_robots[num_pairs:] if num_robots > num_pairs else []
        unassigned_tasks = available_tasks[num_pairs:] if num_tasks > num_pairs else []

        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return assignments

def generate_all_unique_assignments(num_robots: int, num_tasks: int) -> List[Tuple[List[Tuple[int, int]], List[int], List[int]]]:
    """
    Generates all possible unique assignments of robots to tasks.
    Each assignment is a tuple containing:
      1. A list of (robot, task) pairs.
      2. A list of unassigned robots.
      3. A list of unassigned tasks.
    
    Parameters:
        num_robots: Number of available robots.
        num_tasks: Number of tasks to be assigned.
    
    Returns:
        all_assignments: A list of all possible unique assignments, where each assignment is a tuple (assigned_pairs, unassigned_robots, unassigned_tasks).
    """
    all_assignments = []

    # Determine the number of pairs to create (limited by the smaller of num_robots or num_tasks)
    num_pairs = min(num_robots, num_tasks)

    # Generate all possible combinations of robots for the tasks
    robot_combinations = list(combinations(range(num_robots), num_pairs))
    task_combinations = list(combinations(range(num_tasks), num_pairs))

    # Generate all permutations of those combinations
    for robot_comb in robot_combinations:
        for task_comb in task_combinations:
            # Create all permutations for the selected robot and task pairs
            for permuted_robots in permutations(robot_comb):
                for permuted_tasks in permutations(task_comb):
                    # Pair the robots and tasks
                    assigned_pairs = [(permuted_robots[i], permuted_tasks[i]) for i in range(num_pairs)]

                    # Determine unassigned robots and tasks
                    unassigned_robots = [r for r in range(num_robots) if r not in permuted_robots]
                    unassigned_tasks = [t for t in range(num_tasks) if t not in permuted_tasks]

                    all_assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return all_assignments

def generate_high_suitability_assignments(num_robots: int, num_tasks: int, suitability_matrix: List[List[float]], num_assignments: int = 5) -> List[Tuple[List[Tuple[int, int]], List[int], List[int]]]:
    """
    Generates assignments for robots with the highest total suitability across all tasks.
    Each assignment includes the top robots paired with tasks based on suitability.
    
    Parameters:
        num_robots: Number of available robots.
        num_tasks: Number of tasks to be assigned.
        suitability_matrix: A 2D list where element [i][j] represents the suitability of robot i for task j.
        num_assignments: Number of different random assignments to generate.
    
    Returns:
        assignments: A list of high-suitability assignments, where each assignment is a tuple containing:
      - A list of (robot, task) pairs.
      - A list of unassigned robots.
      - A list of unassigned tasks.
    """
    assignments = []
    if num_robots == 1 and num_tasks == 1:
        assigned_pairs = [(0,0)]
        unassigned_robots = []
        unassigned_tasks = []
        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))
        return assignments
    elif num_robots == 1:
        for task_id in range(num_tasks):
            assigned_pairs = [(0, task_id)]
            unassigned_tasks = list(set(range(num_tasks)) - {task_id})
            assignments.append((assigned_pairs, [], unassigned_tasks))
        return assignments
    elif num_tasks == 1:
        for robot_id in range(num_robots):
            assigned_pairs = [(robot_id, 0)]
            unassigned_robots = list(set(range(num_robots)) - {robot_id})
            assignments.append((assigned_pairs, unassigned_robots, []))
        return assignments
        
    num_assignments = min(num_assignments, num_robots * num_tasks * 10)
    

    # Calculate total suitability score for each robot across all tasks
    robot_suitability_scores = [(i, sum(suitability_matrix[i])) for i in range(num_robots)]
    
    # Sort robots by total suitability score in descending order
    sorted_robots = sorted(robot_suitability_scores, key=lambda x: x[1], reverse=True)
    
    # Select the top robots based on total suitability scores
    top_robot_indices = [robot[0] for robot in sorted_robots[:min(num_robots, num_tasks)]]
    
    # Identify robots with the maximum score for each task
    for task in range(num_tasks):
        max_score = max(suitability_matrix[robot][task] for robot in range(num_robots))
        top_task_robots = [robot for robot in range(num_robots) if suitability_matrix[robot][task] == max_score]
        top_robot_indices.extend(top_task_robots)
    
    # Remove duplicates and keep only as many robots as tasks if robots exceed tasks
    top_robot_indices = list(set(top_robot_indices))[:min(num_robots, num_tasks)]

    for _ in range(num_assignments):
        # Randomly sample tasks to pair with the top robots
        available_tasks = random.sample(range(num_tasks), min(num_tasks, num_robots))
        
        # Randomly sample robots to pair with the tasks
        available_robots = random.sample(top_robot_indices, min(num_tasks, num_robots))

        # Pair each selected robot with a randomly chosen task
        assigned_pairs = [(available_robots[i], available_tasks[i]) for i in range(len(available_robots))]
        
        # Determine unassigned robots and tasks
        unassigned_robots = list(set(range(num_robots)) - set(available_robots))
        unassigned_tasks = list(set(range(num_tasks)) - set(available_tasks))
        # Store each assignment as a tuple
        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    #***print(assignments)
    return assignments

In [14]:
def calculate_total_suitability(assignment: List[Tuple[int, int]], suitability_matrix: List[List[float]]) -> float:
    """
    Calculates the total suitability score for a given assignment.
    
    Parameters:
        assignment: A list of (robot, task) pairs representing the assignment.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        total_suitability: The total suitability score for the assignment.
    """
    total_suitability = 0.0
    
    # Sum the suitability ratings for each robot-task pair in the assignment
    for robot, task in assignment:
        total_suitability += suitability_matrix[robot][task]
    
    return total_suitability

def rank_assignments_range(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Ranks the given assignments based on their total suitability scores.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (total_scores, ranking): A tuple where
      1. A list of total suitability scores for each assignment.
      2. A list of indices representing the ranking of the assignments, where the first index corresponds to the
         assignment with the highest total score.
    """
    total_scores = []

    # Calculate total suitability scores for each assignment
    for assignment in assignments:
        total_suitability = sum(suitability_matrix[robot][task] for robot, task in assignment[0])
        total_scores.append(total_suitability)

    # Get the ranking of assignments based on the total scores (higher score is better)
    ranking = sorted(range(len(total_scores)), key=lambda i: total_scores[i], reverse=True)
    #***print (f"ranking: {ranking}")
    return total_scores, ranking

def rank_assignments_borda(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Ranks each assignment from the perspective of each robot based on its suitability for its task in each assignment,
    and uses the Borda count method to aggregate these rankings.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (borda_scores, ranked_assignments): A tuple containing:
      1. A list of Borda scores for each assignment.
      2. A list of indices representing the ranking of the assignments based on Borda scores, where the first index corresponds
         to the assignment with the highest total Borda score.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize Borda scores for each assignment
    borda_scores = [0] * num_assignments

    # For each robot, rank the assignments based on its suitability rating
    for robot in range(num_robots):
        # Collect suitability scores for this robot across all assignments
        robot_scores = []
        
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # If the robot has no task assigned in this assignment, assign it the lowest score
            if assigned_task is None:
                robot_scores.append((assignment_index, -1))  # Using -1 to rank unassigned lower
            else:
                robot_scores.append((assignment_index, suitability_matrix[robot][assigned_task]))

        # Sort robot's scores for all assignments: unassigned (score -1) at the bottom, then by suitability
        robot_scores.sort(key=lambda x: x[1], reverse=True)

        # Assign Borda points based on the ranking
        for rank, (assignment_index, _) in enumerate(robot_scores):
            borda_points = len(robot_scores) - rank - 1  # Borda count: higher ranks get more points
            borda_scores[assignment_index] += borda_points

    # Rank assignments based on total Borda scores (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: borda_scores[i], reverse=True)

    return borda_scores, ranked_assignments

def rank_assignments_approval(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]], threshold: float = 0.5) -> Tuple[List[int], List[int]]:
    """
    Uses approval voting where each robot gives 1 point to assignments where its suitability for its task is above a threshold.
    If a robot's suitability is below the threshold or it is unassigned, it gives 0 points for that assignment.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
        threshold: The suitability threshold above which a robot will approve an assignment (default is 10).
    
    Returns:
        (approval_scores, ranked_assignments): A tuple containing:
      1. A list of approval scores for each assignment.
      2. A list of indices representing the ranking of the assignments based on approval scores.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize approval scores for each assignment
    approval_scores = [0] * num_assignments

    # For each robot, evaluate each assignment based on the suitability threshold
    for robot in range(num_robots):
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # Check if robot's suitability rating for this task is above the threshold
            if assigned_task  and suitability_matrix[robot][assigned_task] > threshold:
                approval_scores[assignment_index] += 1  # Robot approves this assignment

    # Rank assignments based on approval scores (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: approval_scores[i], reverse=True)

    return approval_scores, ranked_assignments

def rank_assignments_majority_judgment(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Uses majority judgment to rank assignments based on robot suitability for assigned tasks.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (assignment_ratings, ranked_assignments): A tuple containing:
      1. A list of median scores for each assignment based on majority judgment.
      2. A list of indices representing the ranking of assignments based on median scores.
    """
    # Define qualitative rating categories mapped to suitability score ranges
    rating_scale = {"Excellent": 3, "Good": 2, "Fair": 1, "Poor": 0}
    rating_thresholds = [(.85, "Excellent"), (.7, "Good"), (.5, "Fair"), (0, "Poor")]

    assignment_ratings = []

    for assignment in assignments:
        ratings = []
        
        for robot, task in assignment[0]:
            # Get the suitability score for the robot-task pair
            suitability_score = suitability_matrix[robot][task]

            # Convert suitability score to qualitative rating
            for threshold, rating in rating_thresholds:
                if suitability_score >= threshold:
                    ratings.append(rating_scale[rating])
                    break
        
        # Calculate the median rating for this assignment
        median_rating = np.median(ratings) if ratings else 0
        assignment_ratings.append(median_rating)

    # Rank assignments based on median ratings
    ranked_assignments = sorted(range(len(assignment_ratings)), key=lambda i: assignment_ratings[i], reverse=True)

    return assignment_ratings, ranked_assignments

def rank_assignments_cumulative_voting(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]], total_votes: int = 10) -> Tuple[List[float], List[int]]:
    """
    Uses cumulative voting to rank assignments, where each robot distributes a fixed number of votes
    across assignments based on suitability scores.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
        total_votes: The total number of votes each robot has to distribute (default is 10).
    
    Returns:
        (cumulative_votes, ranked_assignments): A tuple containing:
      1. A list of cumulative votes for each assignment.
      2. A list of indices representing the ranking of assignments based on cumulative votes.
    """
    num_assignments = len(assignments)
    cumulative_votes = [0] * num_assignments
    if total_votes < num_assignments/2:
        total_votes = num_assignments/2
    
    for robot in range(len(suitability_matrix)):
        # Collect suitability scores for this robot across all assignments
        robot_votes = []
        
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # If robot is unassigned in this assignment, give it a suitability score of 0
            suitability_score = suitability_matrix[robot][assigned_task] if assigned_task is not None else 0
            robot_votes.append((assignment_index, suitability_score))
        
        # Sort robot votes based on suitability scores in descending order
        robot_votes.sort(key=lambda x: x[1], reverse=True)

        # Distribute the total votes among assignments proportional to suitability scores
        total_score = sum(score for _, score in robot_votes if score > 0)
        for assignment_index, score in robot_votes:
            if total_score > 0:
                cumulative_votes[assignment_index] += total_votes * (score / total_score)

    # Rank assignments based on cumulative votes (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: cumulative_votes[i], reverse=True)

    return cumulative_votes, ranked_assignments

def rank_assignments_condorcet_method(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[int], List[int]]:
    """
    Uses the Condorcet method to rank assignments by comparing each assignment in a pairwise manner.
    
    Parameters:
        assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        (pairwise_wins, ranked_assignments): A tuple containing:
      1. A list of pairwise win counts for each assignment.
      2. A list of indices representing the ranking of assignments based on pairwise win counts.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize a matrix to track pairwise wins for each assignment comparison
    pairwise_wins = [0] * num_assignments

    # Compare each pair of assignments
    for i in range(num_assignments):
        for j in range(i + 1, num_assignments):
            # Count the number of robots that prefer assignment i over assignment j and vice versa
            i_wins, j_wins = 0, 0
            
            for robot in range(num_robots):
                # Find the task assigned to the robot in both assignments
                task_i = next((task for r, task in assignments[i][0] if r == robot), None)
                task_j = next((task for r, task in assignments[j][0] if r == robot), None)
                
                # Get suitability scores (or 0 if unassigned)
                suitability_i = suitability_matrix[robot][task_i] if task_i is not None else 0
                suitability_j = suitability_matrix[robot][task_j] if task_j is not None else 0

                # Determine which assignment the robot prefers
                if suitability_i > suitability_j:
                    i_wins += 1
                elif suitability_j > suitability_i:
                    j_wins += 1

            # Update pairwise win counts based on robot preferences
            if i_wins > j_wins:
                pairwise_wins[i] += 1
            elif j_wins > i_wins:
                pairwise_wins[j] += 1

    # Rank assignments based on pairwise win counts
    ranked_assignments = sorted(range(num_assignments), key=lambda k: pairwise_wins[k], reverse=True)

    return pairwise_wins, ranked_assignments

def check_zero_suitability(assignment: List[Tuple[int, int]], suitability_matrix: List[List[float]]) -> bool:
    """
    Checks if any robot-task pair in the assignment has a suitability rating of 0.
    
    Parameters:
        assignment: A list of (robot, task) pairs representing the assignment.
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
        Bool: True if any robot-task pair in the assignment has a suitability of 0, otherwise False.
    """
    for robot, task in assignment:
        if suitability_matrix[robot][task] == 0:
            return True  # Found a zero suitability rating
    
    return False  # No zero suitability ratings found

In [15]:
def cbba_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses a two-phase Consensus-Based Bundle Algorithm (CBBA) for task allocation.
    Each robot is assigned to one task, and each task is assigned to one robot.
    
    Parameters:
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        final_assignment: A list of (robot, task) pairs representing the final allocation.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])

    # Initialize assignment variables
    robot_bundles = [[] for _ in range(num_robots)]  # Bundle of tasks for each robot
    task_bids = [-1] * num_tasks  # Highest bid for each task

    # Phase 1: Bundle Construction
    for robot in range(num_robots):
        # Each robot evaluates each task for inclusion in its bundle
        for task in range(num_tasks):
            bid = suitability_matrix[robot][task]
            if bid > task_bids[task]:
                task_bids[task] = bid
            robot_bundles[robot].append(task)
    
    not_assigned = list(range(num_robots))
    # Phase 2: Conflict Resolution
    updated = True
    while updated:
        updated = False
        
        for robot in range(num_robots):
            if not robot_bundles[robot] or robot not in not_assigned:
                continue  # Skip robots with no tasks in their bundle
                
            # Sort the robot's bundle in descending order of suitability scores
            robot_bundles[robot].sort(key=lambda task: suitability_matrix[robot][task], reverse=True)
            # Iterate through tasks in the robot's bundle
            for task in robot_bundles[robot][:]:
                # If the robot has the highest unique bid for this task, assign it
                highest_bid = max(suitability_matrix[competing_robot][task] for competing_robot in not_assigned)
                if suitability_matrix[robot][task] == highest_bid:
                    updated = True
                    task_bids[task] = suitability_matrix[robot][task]
                    # Clear remaining tasks from this robot's bundle
                    robot_bundles[robot] = [task]
                    not_assigned.remove(robot)
                    for r in range(num_robots):
                        if r != robot and task in robot_bundles[r]:
                            robot_bundles[r].remove(task)
                    break

        if len(not_assigned) == 0:
            updated = False
                    
    final_assignment = [(index, bundle[0]) for index, bundle in enumerate(robot_bundles) if bundle and suitability_matrix[index][bundle[0]] != 0]

    return final_assignment

def ssia_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses the Sequential Single-Item Auction (SSIA) for task allocation.
    
    Parameters:
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        assignments: A list of (robot, task) pairs representing the final allocation.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])
    #**print(f"Suitability Matrix Issue: {suitability_matrix}")
    #**print(f"Number of Robots: {num_robots}, Number of Tasks: {num_tasks}")

    
    # Initialize assignment list to store (robot, task) pairs
    assignments = []
    assigned_robots = set()  # Track robots that have already been assigned
    
    # Auction tasks sequentially
    for task in range(num_tasks):
        # NOTE: I can fix it by making the highest_bid negative infinity but I want to see why the suitability matrix is all negative
        highest_bid = -1
        winning_robot = -1

        # Robots bid for the current task
        for robot in range(num_robots):
            if robot in assigned_robots:
                continue  # Skip if the robot is already assigned
            bid = suitability_matrix[robot][task]
            if bid > highest_bid:
                # check if the bid is positive because we are getting an empty assigned robots dictionary
                #**print(f"Robot {robot} bids {bid} for Task {task}")
                highest_bid = bid
                winning_robot = robot
        
        # Assign the task to the robot with the highest bid if any bid is positive
        # if highest_bid >= 0:
        if winning_robot != -1 and highest_bid > 0:
            assignments.append((winning_robot, task))
            assigned_robots.add(winning_robot)  # Mark this robot as assigned
    
    return assignments

def ilp_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses Integer Linear Programming (ILP) to maximize suitability-based task allocation.
    
    Parameters:
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        assignment: A list of (robot, task) pairs representing the final assignment.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])

    # Define the ILP problem
    problem = pulp.LpProblem("TaskAssignment", pulp.LpMaximize)

    # Define binary decision variables x_ij where x_ij = 1 if robot i is assigned to task j, else 0
    x = [[pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(num_tasks)] for i in range(num_robots)]

    # Objective: Maximize total suitability score
    problem += pulp.lpSum(suitability_matrix[i][j] * x[i][j] for i in range(num_robots) for j in range(num_tasks))

    # Constraint: Each task can be assigned to at most one robot
    for j in range(num_tasks):
        problem += pulp.lpSum(x[i][j] for i in range(num_robots)) <= 1, f"Task_{j}_Assignment"

    # Constraint: Each robot can be assigned to at most one task
    for i in range(num_robots):
        problem += pulp.lpSum(x[i][j] for j in range(num_tasks)) <= 1, f"Robot_{i}_Capacity"

    # Solve the problem
    problem.solve(pulp.PULP_CBC_CMD(msg=False))

    # Collect the results
    assignment = []
    for i in range(num_robots):
        for j in range(num_tasks):
            if pulp.value(x[i][j]) == 1:
                assignment.append((i, j))

    return assignment

In [16]:
import numpy as np
from typing import Callable, List, Tuple

# Type: a candidate is a list of (robot_idx, task_idx) pairs
Assignment = List[Tuple[int, int]]

def generate_candidates_perturb_and_map(
    S: np.ndarray,
    K: int,
    solve_fn: Callable[[np.ndarray], Assignment],
    noise: str = "gumbel",       # "gumbel" or "gaussian"
    scale: float = 0.10,         # noise strength relative to (S.max - S.min)
    anneal: bool = True,         # gradually reduce noise to mix diversity + quality
    max_tries: int = None,       # attempts to collect K unique candidates
    seed: int = None,
    dedup: bool = True,
) -> List[Assignment]:
    """
    Perturb-and-MAP (a.k.a. Perturb-and-Match) candidate generator.
    - S: suitability matrix (R x T), raw or normalized
    - K: desired number of candidates
    - solve_fn: a function that solves max-assignment on a matrix and returns [(i,j), ...]
                e.g. `lambda M: jv_task_allocation(M)[0]` if your JV returns (pairs, score, time)
    - noise: "gumbel" (recommended) or "gaussian"
    - scale: noise strength as a fraction of (S.max - S.min)
    - anneal: if True, noise scale decreases over iterations (diversity early, quality later)
    - dedup: ensure unique candidates
    """
    rng = np.random.default_rng(seed)
    R, T = S.shape
    spread = float(S.max() - S.min() + 1e-9)
    base_sigma = scale * spread
    max_tries = max_tries or (5 * K)

    def _sample_noise(shape, sigma):
        if noise == "gumbel":
            # Gumbel(0, beta). std ≈ 1.28255*beta → pick beta so that std ≈ sigma
            beta = sigma / 1.28255 if sigma > 0 else 0.0
            return rng.gumbel(loc=0.0, scale=max(1e-12, beta), size=shape)
        elif noise == "gaussian":
            return rng.normal(loc=0.0, scale=max(1e-12, sigma), size=shape)
        else:
            raise ValueError("noise must be 'gumbel' or 'gaussian'")

    # storage
    candidates: List[Assignment] = []
    seen = set()

    tries = 0
    i = 0
    while len(candidates) < K and tries < max_tries:
        tries += 1
        # anneal noise from base_sigma → base_sigma/5 (heuristic)
        if anneal and K > 1:
            frac = i / max(1, K - 1)  # 0..1
            sigma = base_sigma * (0.2 + 0.8 * (1.0 - frac))
        else:
            sigma = base_sigma

        N = _sample_noise((R, T), sigma)
        S_pert = S + N

        # Solve max assignment on perturbed scores
        assignment: Assignment = solve_fn(S_pert)

        # Dedup by making a canonical key
        if dedup:
            key = tuple(sorted(assignment))
            if key in seen:
                continue
            seen.add(key)

        candidates.append(assignment)
        i += 1

    return candidates


In [17]:
def print_robots(robots: List[CapabilityProfile]):
    for robot in robots:
        print(robot.robot_id)
        
def print_tasks(tasks: List[TaskDescription]):
    for task in tasks:
        print(task.task_id)

def calculate_suitability_matrix(robots: List[CapabilityProfile], tasks: List[TaskDescription], suitability_method: str) -> np.ndarray:
    """
    Calculates the suitability matrix for the given robots and tasks.
    
    Parameters:
        robots: List of robot profiles.
        tasks: List of task descriptions.
        suitability_method: The name of the suitability evaluation function.
    
    Returns:
        suitability_matrix: A 2D numpy array representing the suitability scores of each robot-task pair.
    """
    num_robots = len(robots)
    num_tasks = len(tasks)
    suitability_matrix = np.zeros((num_robots, num_tasks))

    # Evaluate suitability of each robot for each task
    for i, robot in enumerate(robots):
        for j, task in enumerate(tasks):
            suitability_score = globals()[suitability_method](robot, task)
            # NOTE: IMPORTANT can suitability_matrix hold robot.robot_id and task.task_id so that these values can be identified? maybe each score [i][j] can relate to a (robot, task) tuple that can be accessed like below?
            # suitability_matrix[i][j] = (suitability_score, (robot.robot_id, task.task_id)) # this way every suitability_score in the matrix is now associated with an unchanging pair and we don't have to interpret the indices
            suitability_matrix[i][j] = suitability_score
#             print(f"Suitability of {robot.robot_id} for {task.task_id}: {suitability_score:.2f}")

    # Print the suitability matrix
#     print("\nSuitability Matrix:")
    # print(suitability_matrix)
            
    return suitability_matrix

def assign_tasks_with_voting(robots: List[CapabilityProfile], tasks: List[TaskDescription], suitability_matrix: np.ndarray, num_candidates: int, voting_method: str):
    """
    Assigns tasks to robots using random assignment and ranks the assignments using the specified voting method.
    
    Parameters:
        robots: List of robot profiles.
        tasks: List of task descriptions.
        suitability_matrix: A 2D numpy array with suitability scores for each robot-task pair.
        num_candidates: Number of candidate assignments to generate.
        voting_method: The name of the voting function.
    
    Returns:
        (best_assignment, best_score, length): The best assignment, its suitability score, and the time taken for the voting process.
    """
    num_robots = len(robots)
    num_tasks = len(tasks)

    # Display generated robots and tasks
#     print("\nGenerated Robot Profiles:")
#     for robot in robots:
#         print(robot)

#     print("\nGenerated Task Descriptions:")
#     for task in tasks:
#         print(task)
    
#     random_assignments = generate_random_assignments(num_robots, num_tasks, num_candidates)
    def map_with_jv(S_mat):
        pairs = jv_task_allocation(S_mat)  # adjust to match your JV return
        return pairs

    candidate_assignments = generate_candidates_perturb_and_map(
        S=suitability_matrix,
        K=num_candidates,
        solve_fn=map_with_jv,
        noise="gumbel",
        scale=0.10,
        anneal=True,
        seed=None
    )

    # Reformat to your (assignment, unassigned robots, unassigned tasks) structure
    random_assignments = [ (cand, [], []) for cand in candidate_assignments ]

    # random_assignments = generate_high_suitability_assignments(num_robots, num_tasks, suitability_matrix, num_candidates)

#     print("\nRandom Complete Assignments of Robots to Tasks:")
#     for i, assignment in enumerate(random_assignments):
#         print(f"Assignment {i+1}: {assignment}")
#         total_score = calculate_total_suitability(assignment[0], suitability_matrix)
#         print(f"Total Suitability Score for the Assignment: {total_score:.2f}")
    
    start = time.perf_counter_ns()
    # NOTE: are assignment rankings using robot_ids from the capability profiles?
    total_scores, assignment_ranking = globals()[voting_method](random_assignments, suitability_matrix)
    end = time.perf_counter_ns()
    length = (end - start) / 1000.0
    
#     print("\nTotal Suitability Scores for Each Assignment:")
#     for i, score in enumerate(total_scores):
#         print(f"Assignment {i+1}: {score:.2f}")

#     print("\nRanking of Assignments (from best to worst):")
#     for rank, assignment_index in enumerate(assignment_ranking):
#         print(f"Rank {rank+1}: Assignment {assignment_index+1}")

    best_ranking = 0
    while(check_zero_suitability(random_assignments[assignment_ranking[best_ranking]][0], suitability_matrix) and best_ranking < len(assignment_ranking)-1):
        best_ranking += 1
    if best_ranking == num_candidates-1:
        best_ranking = 0
    
#     print(best_ranking)
#     print(random_assignments[assignment_ranking[best_ranking]])
    # best_assignment = [[(robot, task), (robot, task), ...], [unassigned robots], [unassigned tasks]]
    best_assignment = random_assignments[assignment_ranking[best_ranking]]
    filtered_best_assignments = ([],[],[])
    for robot_id, task_id in best_assignment[0]:
            if suitability_matrix[robot_id][task_id] == 0:
                filtered_best_assignments[1].append(robot_id)
                filtered_best_assignments[2].append(task_id)
            else:
                filtered_best_assignments[0].append((robot_id, task_id))
    print(f"Best assignment {filtered_best_assignments}")

    best_score = calculate_total_suitability(filtered_best_assignments[0], suitability_matrix)
    # print(f"BEST ASSIGNMENT: {best_assignment}")
    # print(f"BEST ASSIGNMENT[0]: {best_assignment[0]}")

    # returns the best assignment of robots to tasks and the total suitability score of the best assignment
    #print(f"Score of the Best Assignment: {best_score:.2f}")
    #print(f"Time taken for the voting process: {length:.2f} microseconds")
    return filtered_best_assignments, best_score, length

def assign_tasks_with_method(
    allocation_method: Callable[[List[List[float]]], List[Tuple[int, int]]],
    suitability_matrix: List[List[float]]
):
    """
    Assigns tasks using a specified allocation method and returns the allocation details.
    
    Parameters:
        allocation_method: The function used for task allocation (e.g., `cbba_task_allocation`, `ssia_task_allocation`, `ilp_task_allocation`).
        suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
        (assignment, unassigned_robots, unassigned_tasks, total_score, allocation_time): A tuple containing:
      1. A list of assigned (robot, task) pairs.
      2. A list of unassigned robot indices.
      3. A list of unassigned task indices.
      4. The total suitability score of the assignment.
      5. The time taken for the allocation (in microseconds).
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])
    
    # Start timing
    start_time = time.perf_counter_ns()
    
    # Get the assignment using the specified allocation method
    assignment = allocation_method(suitability_matrix)
    
    # End timing
    end_time = time.perf_counter_ns()
    allocation_time = (end_time - start_time) / 1000.0  # Convert nanoseconds to microseconds
    
    # Calculate total suitability score for the assignment
    total_score = calculate_total_suitability(assignment, suitability_matrix)
    
    # Determine unassigned robots and tasks
    assigned_robots = {robot for robot, _ in assignment}
    assigned_tasks = {task for _, task in assignment}
    
    unassigned_robots = [robot for robot in range(num_robots) if robot not in assigned_robots]
    unassigned_tasks = [task for task in range(num_tasks) if task not in assigned_tasks]

    #**print(f"Total Suitability Score: {total_score:.2f}")
    #**print(f"Time taken for allocation: {allocation_time:.2f} microseconds")
    
    return (assignment, unassigned_robots, unassigned_tasks), total_score, allocation_time


In [18]:
def simulate_time_step(
    robots: List[CapabilityProfile],
    tasks: List[TaskDescription],
    unassigned_robots: List[str],
    unassigned_tasks: List[str],
    suitability_method: str,
    occupied_locations: set,
    start_positions: dict,
    goal_positions: dict,
    time_step: float = 1.0,
    total_reward: float = 0.0,
    total_success: int = 0
) -> Tuple[int, float, int]:
    """
    Simulates a single time step, updating robot positions, task progress, and handling failures.

    Parameters:
        robots: List of all robots.
        tasks: List of all tasks.
        time_step: The time increment for the simulation step.
        total_reward: Accumulated reward from successfully completed tasks.

    Returns:
        (count, total_reward, total_success): A count of unassigned robots and the updated total reward.
    """
    count = 0  # Count of unassigned robots
    # Iterate through all robots to update their positions and tasks
    for robot in robots:
        if robot.assigned: # Check that robot is assigned
            if robot.current_task is not None: # Check that all assigned robots have a task
                # Get the assigned task
                task = robot.current_task

                # check if there is more path to traverse for the robot
                if robot.current_path and len(robot.current_path) > 1:

                    next_position = robot.current_path[1] # gives (x, y) coordinate of next step in path
                    occupied_locations.discard(robot.location) # remove current location from occupied set
                    robot.location = next_position # update location
                    # start position for this robot should be replaced, if not then must index by ID
                    start_positions[robot.robot_id] = next_position
                    occupied_locations.add(next_position) # update occupied set with robots new current location
                    robot.current_path.pop(0) # Move the robot one space by removing the first element from the robots path
                    robot.remaining_distance = len(robot.current_path) - 1 # recalculate the remaining distance and new location

                    if robot.remaining_distance <= 1:
                        # The robot has reached the task
                        robot.remaining_distance = 0
                        # this wont work here because once the robot reaches the task it will just keep resetting the time on task, I put it down below after completing the task
                        # robot.time_on_task = 0  # Reset time on task so it can begin work (time on task is a counter for how long it takes to complete a task)


                    if robot.battery_life == 0: 
                        # print(f"Robot {robot.robot_id} failed to reach task {task.task_id} due to mechanical failure.")
                        # get task id and task index
                        unassign_task_from_robot(robot, tasks=tasks, unassigned_robots=unassigned_robots, unassigned_tasks=unassigned_tasks)
                        continue

                # If the robot is at the task, increment time on task
                if robot.remaining_distance == 0:
                    #robot.time_on_task += time_step
                    robot.battery_life -= time_step
                    task.time_left -= time_step
                    # suitability = globals()[suitability_method](robot, task)
                    # failure_probability = 1 / (100 * (suitability + 1))  # Higher suitability, lower failure rate
                    # Check if the task is completed
                    if task.time_left <= 0:
                        # Mark task as completed
                        total_reward += task.reward
                        total_success += 1
                        robot.current_task = None
                        robot.tasks_successful += 1
                        robot.assigned = False
                        robot.time_on_task = 0  # Reset time on task so it can begin work (time on task is a counter for how long it takes to complete a task)
                        # move it to unassigned robots list with check
                        rid = robot.robot_id
                        if rid not in unassigned_robots:
                            unassigned_robots.append(rid)
                        print(f"Robot {robot.robot_id} completed task {task.task_id}")
                        tasks.remove(task)
                    elif robot.battery_life == 0:
                        if (task.reset_progress): # Only resets progress for certain tasks
                            task.time_left = task.time_to_complete
                        else:
                            task.time_to_complete = task.time_left
                        unassign_task_from_robot(robot, tasks=tasks, unassigned_robots=unassigned_robots, unassigned_tasks=unassigned_tasks)
                    elif task.performance_metrics == "safety compliance":
                        if robot.safety_features:
                            matched_safety = sum(safety in robot.safety_features for safety in task.safety_protocols)
                            safety_score = matched_safety/len(task.safety_protocols)
                            if safety_score < 0.75:
                                if (random.random() > 0.75):
                                    unassign_task_from_robot(robot, tasks=tasks, unassigned_robots=unassigned_robots, unassigned_tasks=unassigned_tasks)
                        else:
                            unassign_task_from_robot(robot, tasks=tasks, unassigned_robots=unassigned_robots, unassigned_tasks=unassigned_tasks)
           
            else:
                print(f"Assigned robot {robot.robot_id} has no current task")
        else:
            count += 1
    return count, total_reward, total_success

def unassign_task_from_robot(robot: CapabilityProfile, tasks: List[TaskDescription], unassigned_robots: List[str], unassigned_tasks: List[str]):
    task_id = robot.current_task.task_id
    t_index = [task.task_id for task in tasks].index(task_id)
    # unassign task
    tasks[t_index].assigned_robot = None
    tasks[t_index].assigned = False
    robot.assigned = False

    # move it to unassigned tasks list with check
    tid = tasks[t_index].task_id
    if tid not in unassigned_tasks:
        unassigned_tasks.append(tid)
    # unassign robot
    robot.current_task = None
    # move it to unassigned robots list with check
    rid = robot.robot_id
    if rid not in unassigned_robots:
        unassigned_robots.append(rid)

def add_new_tasks(tasks: List[TaskDescription], unassigned_tasks: List[str], task_max_id: int, new_task_count: int, total_tasks: int, grid: List[List[int]], occupied_locations: set) -> Tuple[int, int]:
    """
    This function generates new unassigned tasks with random descriptions and adds them to the tasks list.
    It also updates the unassigned tasks list and occupied locations set.
    
    Parameters:
        tasks: List of existing task descriptions.
        unassigned_tasks: List of unassigned task IDs.
        task_max_id: The current maximum task ID to ensure unique IDs for new tasks.
        new_task_count: The number of new tasks to add.
        grid: The grid representing the environment where tasks are located.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        task_max_id: The updated maximum task ID after adding new tasks.
        total_tasks: The updated total number of tasks in the system.
    """
    for _ in range(new_task_count):
        task_id = f"T{task_max_id}"
        task_max_id += 1
        total_tasks += 1
        new_task = generate_random_task_description(task_id, grid, occupied_locations, tasks)
        tasks.append(new_task)
        unassigned_tasks.append(task_id)
    return task_max_id, total_tasks

def add_new_tasks_strict(tasks: List[TaskDescription], unassigned_tasks: List[str], task_max_id: int, new_task_count: int, total_tasks: int, grid: List[List[int]], occupied_locations: set) -> Tuple[int, int]:
    """
    This function generates new unassigned tasks with random descriptions and adds them to the tasks list.
    It also updates the unassigned tasks list and occupied locations set.
    
    Parameters:
        tasks: List of existing task descriptions.
        unassigned_tasks: List of unassigned task IDs.
        task_max_id: The current maximum task ID to ensure unique IDs for new tasks.
        new_task_count: The number of new tasks to add.
        grid: The grid representing the environment where tasks are located.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        task_max_id: The updated maximum task ID after adding new tasks.
        total_tasks: The updated total number of tasks in the system.
    """
    for _ in range(new_task_count):
        task_id = f"T{task_max_id}"
        task_max_id += 1
        total_tasks += 1
        new_task = generate_random_task_description_strict(task_id, grid, occupied_locations, tasks)
        tasks.append(new_task)
        unassigned_tasks.append(task_id)
    return task_max_id, total_tasks

def add_new_robots(robots: List[CapabilityProfile], unassigned_robots: List[str], robot_max_id: int, new_robot_count: int, grid: List[List[int]], occupied_locations: set) -> int:
    """
    This function generates new unassigned robots with random profiles and adds them to the robots list.
    It also updates the unassigned robots list and occupied locations set.
    
    Parameters:
        robots: List of existing robot profiles.
        unassigned_robots: List of unassigned robot IDs.
        robot_max_id: The current maximum robot ID to ensure unique IDs for new robots.
        new_robot_count: The number of new robots to add.
        grid: The grid representing the environment where robots operate.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        robot_max_id: The updated maximum robot ID after adding new robots.
        """
    for _ in range(new_robot_count):
        robot_id = f"R{robot_max_id}"
        robot_max_id += 1
        new_robot = generate_random_robot_profile(robot_id, grid, occupied_locations)
        robots.append(new_robot)
        unassigned_robots.append(robot_id)
        occupied_locations.add(new_robot.location)
        # NOTE: do not update start_positions until robots are given a task because they will be in a new location each time they are assigned a task if they get reassigned and CBS recalculates paths
        # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
    return robot_max_id

def add_new_robots_strict(robots: List[CapabilityProfile], unassigned_robots: List[str], robot_max_id: int, new_robot_count: int, grid: List[List[int]], occupied_locations: set) -> int:
    """
    This function generates new unassigned robots with random profiles and adds them to the robots list.
    It also updates the unassigned robots list and occupied locations set.
    
    Parameters:
        robots: List of existing robot profiles.
        unassigned_robots: List of unassigned robot IDs.
        robot_max_id: The current maximum robot ID to ensure unique IDs for new robots.
        new_robot_count: The number of new robots to add.
        grid: The grid representing the environment where robots operate.
        occupied_locations: A set of currently occupied locations to avoid conflicts.
        
    Returns:
        robot_max_id: The updated maximum robot ID after adding new robots.
        """
    for _ in range(new_robot_count):
        robot_id = f"R{robot_max_id}"
        robot_max_id += 1
        new_robot = generate_random_robot_profile_strict(robot_id, grid, occupied_locations)
        robots.append(new_robot)
        unassigned_robots.append(robot_id)
        occupied_locations.add(new_robot.location)
        # NOTE: do not update start_positions until robots are given a task because they will be in a new location each time they are assigned a task if they get reassigned and CBS recalculates paths
        # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
    return robot_max_id


def remove_random_robots(robots: List[CapabilityProfile], tasks: List[TaskDescription], unassigned_robots: List[str], unassigned_tasks: List[str], count: int, occupied_locations: set, start_positions: dict, goal_positions: dict):
    """
    This function selects a specified number of robots to remove from the robots list.
    It updates the unassigned robots and tasks lists, as well as the occupied locations set.
    
    Parameters:
        robots: List of existing robot profiles.
        tasks: List of existing task descriptions.
        unassigned_robots: List of unassigned robot IDs.
        unassigned_tasks: List of unassigned task IDs.
        count: The number of robots to remove.
        occupied_locations: A set of currently occupied locations to update.
        start_positions: A dictionary mapping robot IDs to their start positions.
        goal_positions: A dictionary mapping robot IDs to their goal positions.
        
    Returns:
        None
    """
    for _ in range(min(count, (len(robots) - 1))): # Make sure theres always at least one robot so CBS doesnt break
        robot_to_remove = random.choice(robots)
        occupied_locations.discard(robot_to_remove.location)
        if not robot_to_remove.assigned and robot_to_remove.current_task == None:
            if robot_to_remove.robot_id in unassigned_robots:
                # If the robot is unassigned, just remove it from the list
                unassigned_robots.remove(robot_to_remove.robot_id)
        else:
            task_id = robot_to_remove.current_task.task_id
            t_index = [task.task_id for task in tasks].index(task_id)
            tasks[t_index].assigned_robot = None
            tasks[t_index].assigned = False
            if task_id not in unassigned_tasks:
            # If the task is not already in the unassigned tasks list, add it
                unassigned_tasks.append(task_id)
            # in the case an assigned robot is removed we must update the start_positions and goal_positions lists to remove both locations so they are no longer used in the CBS pathfinding
            del start_positions[robot_to_remove.robot_id]
            del goal_positions[robot_to_remove.robot_id]
            # check if the robot is in the unassigned robots list and remove it
            if robot_to_remove.robot_id in unassigned_robots:
                # If the robot is unassigned, just remove it from the list
                unassigned_robots.remove(robot_to_remove.robot_id)
        robots.remove(robot_to_remove)
#         print(f"Robot {robot_to_remove.robot_id} left the system. It attempted {robot_to_remove.tasks_attempted} tasks and successfully completed {robot_to_remove.tasks_successful} of them.")

def reassign_robots_to_tasks(
        robots: List[CapabilityProfile], 
        tasks: List[TaskDescription], 
        num_candidates: int, 
        voting_method: str, 
        suitability_method: str, 
        unassigned_robots: List[str], 
        unassigned_tasks: List[str], 
        start_positions: dict, 
        goal_positions: dict,
        inertia_threshold: float = 0.1):
    """
    Reassigns unassigned robots to unassigned tasks using a voting method.
    Parameters:
        robots: List of all robot profiles.
        tasks: List of all task descriptions.
        num_candidates: Number of candidate assignments to generate.
        voting_method: The name of the voting function to use for ranking assignments.
        suitability_method: The name of the suitability evaluation function.
        unassigned_robots: List of unassigned robot IDs.
        unassigned_tasks: List of unassigned task IDs.
        start_positions: Dictionary mapping robot IDs to their start positions.
        goal_positions: Dictionary mapping robot IDs to their goal positions.
        Inertia threshold: minimum improvement in suitability required to steal an already‐assigned task.

    Returns:
        return_assignments: Dictionary mapping robot IDs to assigned task IDs.
        unassigned_robots: List of unassigned robot IDs after reassignment.
        unassigned_tasks: List of unassigned task IDs after reassignment.
        score: Total suitability score of the best assignment.
        length: Time taken for the voting process in microseconds.
    """
    urobots = [robot for robot in robots if not robot.assigned]
    utasks = [task for task in tasks if not task.assigned]

    suitability_matrix = calculate_suitability_matrix(urobots, utasks, suitability_method)
    output, score, length = assign_tasks_with_voting(
        urobots, utasks, suitability_matrix, num_candidates, voting_method)

    # this assigned_pairs only contains the unassigned robots and tasks, I may have to pass in the actual assigned_pairs to update it
    assigned_pairs = output[0] # list of tuples
    return_assignments = {}
    unassigned_robots = [urobots[i].robot_id for i in output[1]]
    unassigned_tasks = [utasks[j].task_id for j in output[2]]
    for (robot_idx, task_idx) in assigned_pairs:
        # NOTE: THIS IS RIGHT, indexes into the filtered list of assigned pairs with the unassigned robots and tasks
        # print(f"UROBOT: {urobots[robot_idx].robot_id} | UTASK: {utasks[task_idx].task_id}")
        r = urobots[robot_idx]
        t = utasks[task_idx]
        r.current_task = t
        r.tasks_attempted += 1
        t.assigned_robot = r
        r.assigned = True
        t.assigned = True
        # update start and goal positions when robots are assigned
        start_positions[r.robot_id] = r.location
        goal_positions[r.robot_id] = t.location
        return_assignments[r.robot_id] = t.task_id
        # return_assignments.update({urobots[robot_idx].robot_id : utasks[task_idx].task_id})

    # Check for stealing tasks from already assigned robots
    free_robots = [r for r in robots if not r.assigned]
    for task in tasks:
        current = task.assigned_robot
        if current is None:
            continue
        current_suitability = globals()[suitability_method](current, task)
        # find the best free robot for this task
        best, best_suit = None, current_suitability
        for r in free_robots:
            s = globals()[suitability_method](r, task)
            if s > best_suit:
                best, best_suit = r, s
        # Inertia check: if the best free robot's suitability is not significantly better, skip stealing
        if best and (best_suit - current_suitability) >= inertia_threshold:
            # unassign the current robot from the task
            current.current_task = None
            current.assigned = False
            if current.robot_id not in unassigned_robots:
                unassigned_robots.append(current.robot_id)
            # update the task's assigned robot
            best.current_task = task
            best.assigned = True
            best.tasks_attempted += 1
            task.assigned_robot = best
            start_positions[best.robot_id] = best.location
            goal_positions[best.robot_id] = task.location
            # remove from free list and unassigned robots
            free_robots.remove(best)
            if best.robot_id in unassigned_robots:
                unassigned_robots.remove(best.robot_id)
    
    print(f"Reassign Score: {score}, Reassign Length: {length}")

    return return_assignments, unassigned_robots, unassigned_tasks, score, length

def reassign_robots_to_tasks_with_method(
        robots: List[CapabilityProfile], 
        tasks: List[TaskDescription], 
        num_candidates: int, 
        voting_method: str, 
        suitability_method: str, 
        unassigned_robots: List[str], 
        unassigned_tasks: List[str], 
        allocation_method: Callable[[List[List[float]]], List[Tuple[int, int]]], 
        start_positions: dict, 
        goal_positions: dict,
        inertia_threshold: float = 0.1):
    """
    Reassigns unassigned robots to unassigned tasks using a specified allocation method.
    Parameters:
        robots: List of all robot profiles.
        tasks: List of all task descriptions.
        num_candidates: Number of candidate assignments to generate.
        voting_method: The name of the voting function to use for ranking assignments.
        suitability_method: The name of the suitability evaluation function.
        unassigned_robots: List of unassigned robot IDs.
        unassigned_tasks: List of unassigned task IDs.
        allocation_method: The function used for task allocation (e.g., `cbba_task_allocation`, `ssia_task_allocation`, `ilp_task_allocation`).
        start_positions: Dictionary mapping robot IDs to their start positions.
        goal_positions: Dictionary mapping robot IDs to their goal positions.
        Inertia threshold: minimum improvement in suitability required to steal an already‐assigned task.
        
    Returns:
        return_assignments: Dictionary mapping robot IDs to assigned task IDs.
        unassigned_robots: List of unassigned robot IDs after reassignment.
        unassigned_tasks: List of unassigned task IDs after reassignment.
        score: Total suitability score of the best assignment.
        length: Time taken for the voting process in microseconds.
    """
    urobots = [robot for robot in robots if not robot.assigned]
    utasks  = [task for task in tasks if not task.assigned]

    suitability_matrix = calculate_suitability_matrix(urobots, utasks, suitability_method)
    output, score, length = assign_tasks_with_method(allocation_method, suitability_matrix)

    assigned_pairs = output[0]
    return_assignments = {}
    unassigned_robots = [urobots[i].robot_id for i in output[1]]
    unassigned_tasks = [utasks[j].task_id for j in output[2]]
    for (robot_idx, task_idx) in assigned_pairs:
        # print(f"UROBOT: {urobots[robot_idx].robot_id} | UTASK: {utasks[task_idx].task_id}")
        r = urobots[robot_idx]
        t = utasks[task_idx]
        r.current_task = t
        r.tasks_attempted += 1
        t.assigned_robot = r
        r.assigned = True
        t.assigned = True
        # update start and goal positions when robots are assigned
        start_positions[r.robot_id] = r.location
        goal_positions[r.robot_id] = t.location
        return_assignments[r.robot_id] = t.task_id
    
    # Check for stealing tasks from already assigned robots
    free_robots = [r for r in robots if not r.assigned]
    for task in tasks:
        current = task.assigned_robot
        if current is None:
            continue
        current_suitability = globals()[suitability_method](current, task)
        # find the best free robot for this task
        best, best_suit = None, current_suitability
        for r in free_robots:
            s = globals()[suitability_method](r, task)
            if s > best_suit:
                best, best_suit = r, s
        # Inertia check: if the best free robot's suitability is not significantly better, skip stealing
        if best and (best_suit - current_suitability) >= inertia_threshold:
            # unassign the current robot from the task
            current.current_task = None
            current.assigned = False
            if current.robot_id not in unassigned_robots:
                unassigned_robots.append(current.robot_id)
            # update the task's assigned robot
            best.current_task = task
            best.assigned = True
            best.tasks_attempted += 1
            task.assigned_robot = best
            start_positions[best.robot_id] = best.location
            goal_positions[best.robot_id] = task.location
            # remove from free list and unassigned robots
            free_robots.remove(best)
            if best.robot_id in unassigned_robots:
                unassigned_robots.remove(best.robot_id)

    print(f"Reassign Score: {score}, Reassign Length: {length}")

    return return_assignments, unassigned_robots, unassigned_tasks, score, length
    
def create_obstacle_list(grid):
    """Create a reusable obstacle list from the grid for fast lookups"""
    obstacle_list = []
    for r in range(len(grid)):
        for c in range(len(grid[r])):
            if grid[r][c] == 1:
                obstacle_list.append((r, c)) # store all obstacles in a list
    return obstacle_list

def build_cbs_agents(robots, start_positions, goal_positions):
    """
    Return a fresh list of {'name','start','goal'} dicts for CBS,
    based on whatever robots are currently marked assigned.
    """
    return [
        {
            'name': r.robot_id,
            'start': start_positions[r.robot_id],
            'goal':  goal_positions[r.robot_id],
        }
        for r in robots
        if r.assigned and r.current_task is not None
    ]

In [33]:
import csv

map_file = r"test_small_open.map"
grid = load_map(map_file) 

robot_sizes = [10, 50, 100, 500, 1000, 5000]

num_repititions = 5
if __name__ == "__main__":
    with open('VotingData.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Method', 'Robots', 'Tasks', 'Candidates', 'Score', 'Task Normalized Score', 'Score Density', 'Length'])
        for num_robots in robot_sizes: 
            task_sizes = [
                int(num_robots * 0.5),
                int(num_robots * 0.75),
                num_robots,  
                int(num_robots * 1.25),
                int(num_robots * 1.5)
            ]
            for num_tasks in task_sizes:
                for num_candidates in [10, 25, 50, 100]:
                    for rep in range(num_repititions):     
                        robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(num_robots)]
                        tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(num_tasks)]
                        suitability_matrix = calculate_suitability_matrix(robots, tasks, "evaluate_suitability_new")
                            
                        voting_methods = [
                            "rank_assignments_borda", 
                            "rank_assignments_approval", 
                            "rank_assignments_majority_judgment", 
                            "rank_assignments_cumulative_voting", 
                            "rank_assignments_condorcet_method", 
                            "rank_assignments_range"
                            ]

                        for method in voting_methods:
                            output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, num_candidates, method)
                            assigned_count = len(output[0]) if output and output[0] else 0 # nuber of pairs in the chosen assignment
                            task_normalized_score = (score / assigned_count) if assigned_count > 0 else 0.0 # per assigned task normalized score
                            score_density = (score / (num_robots * num_tasks)) if (num_robots * num_tasks) > 0 else 0.0
                            writer.writerow([method, num_robots, num_tasks, num_candidates, score, task_normalized_score, score_density, length])
                                
                        allocation_methods = [cbba_task_allocation, ssia_task_allocation, ilp_task_allocation, jv_task_allocation]
                        allocation_names = ["cbba_task_allocation", "ssia_task_allocation", "ilp_task_allocation", "jv_task_allocation"]
                        for method_idx, alloc_name in enumerate(allocation_methods):
                            output, score, length = assign_tasks_with_method(alloc_name, suitability_matrix)
                            assigned_count = len(output[0]) if output and output[0] else 0 # number of pairs in the chosen assignment
                            task_normalized_score = (score / assigned_count) if assigned_count > 0 else 0.0 # per assigned task normalized score
                            score_density = (score / (num_robots * num_tasks)) if (num_robots * num_tasks) > 0 else 0.0
                            writer.writerow([allocation_names[method_idx], num_robots, num_tasks, "", score, task_normalized_score, score_density, length]) # leave num candidates empty bc its not used in optimization methods

Best assignment ([(0, 4), (3, 2), (4, 3), (8, 0), (9, 1)], [], [])
Best assignment ([(0, 4), (3, 2), (7, 1), (8, 0), (9, 3)], [], [])
Best assignment ([(0, 4), (2, 0), (6, 2), (7, 1), (8, 3)], [], [])
Best assignment ([(0, 4), (1, 3), (2, 0), (4, 1), (6, 2)], [], [])
Best assignment ([(0, 4), (3, 3), (4, 1), (7, 2), (8, 0)], [], [])
Best assignment ([(0, 4), (2, 0), (4, 3), (7, 2), (8, 1)], [], [])
Best assignment ([(1, 0), (4, 3), (6, 2), (7, 1), (9, 4)], [], [])
Best assignment ([(2, 0), (4, 3), (5, 4), (6, 2), (7, 1)], [], [])
Best assignment ([(2, 0), (4, 3), (6, 2), (7, 1), (9, 4)], [], [])
Best assignment ([(1, 0), (4, 3), (6, 2), (7, 1), (9, 4)], [], [])
Best assignment ([(0, 4), (4, 3), (6, 2), (7, 1), (9, 0)], [], [])
Best assignment ([(4, 3), (6, 2), (7, 1), (8, 0), (9, 4)], [], [])
Best assignment ([(1, 4), (2, 2), (4, 3), (6, 1), (7, 0)], [], [])
Best assignment ([(0, 2), (1, 0), (4, 3), (6, 1), (7, 4)], [], [])
Best assignment ([(0, 2), (1, 4), (4, 3), (6, 1), (7, 0)], [],

KeyboardInterrupt: 

In [16]:
import psutil

def suitability_all_zero(suitability_matrix):
    return all(value == 0 for row in suitability_matrix for value in row)

def main_simulation(output: tuple[list[tuple[int, int]],list[int],list[int]], robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, grid: List[List[int]], map_dict: dict, suitability_method: str, suitability_matrix: np.ndarray, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    print(f"SUITABILITY METHOD: {suitability_method}")
    
    initial_positions = set()
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = len(tasks)
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions) # use the occcupied positions as the current positions for CBS, this is just as a occupation check, not a start position

    assigned_pairs = output[0]  
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True

    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }
    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    # print(f"ASSIGNED PAIRS: {assigned_pairs}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    print(f"AGENTS LIST: {agents}")

    # Create the input data dictionary for CBS, this will be passed to the CBS planner
    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
        # could possibly fall back on the simple method here if we get a lot of issues
    else:

        # Iterate through the agents and their schedules
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            # Iterate through the schedule (list of dictionaries)
            for point in schedule:
                # Access each entry in the schedule
                # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                # Assign the paths to the robots
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    # Keep track of previous state to not run CBS when there are no changes
    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
 
    # End the simulation if nothing changes for 3 timesteps (hopefully the tasks are done and the robots arent moving)
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        print(f"START POSITIONS: {start_positions}")
        print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        # before each time step, refresh the unassigned robots and tasks lists
        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        # Periodically add new tasks and robots
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks_strict(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots_strict(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        # Periodically remove robots
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1: # Otherwise will break CBS, we need at least one agent for things to run smoothly
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)
 

        # Reassign unassigned robots to unassigned tasks
        # TODO: change this to make sure newly assigned robot is more suitable for the task
        # so we will run it when there are unassigned robots AND a state change, otherwise theres no reason to keep checking
        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(
                robots, tasks, num_candidates, voting_method, suitability_method,
                unassigned_robots, unassigned_tasks, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score
            # regenerate the master assigned robots dictionary
            assigned_robots = {
                r.robot_id: r.current_task.task_id
                for r in robots
                if r.assigned and r.current_task is not None
            }

            print(f"ROBOTS REASSIGNED")
            # print(f"NEW ASSIGNMENTS FROM REASSIGNMENT: {new_assignments}")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")
 
 
        # Update start and goal positions before cbs
        for robot in robots:
            if robot.assigned and robot.current_task:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }
 
        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            # NOTE: use occupied positions as current positions for running CBS, update goal positions whenever needed
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            agents = build_cbs_agents(robots, start_positions, goal_positions)

            print(f"AGENTS LIST: {agents}")
 
            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # if time_steps_unchanged < 3:
            #     time_steps_unchanged += 1
            #     print("No state change, skip CBS...")
            #     print(f"ALL ROBOTS: {len(robots)}")
            #     print(f"ALL TASKS: {len(tasks)}")
            #     print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            #     print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # else:
            #     break

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

def benchmark_simulation(output: tuple[list[tuple[int, int]],list[int],list[int]], robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, grid: List[List[int]], map_dict: dict, suitability_method: str, suitability_matrix: np.ndarray, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    start_time = time.time()
    main_simulation(output, robots, tasks, num_candidates, voting_method, grid, map_dict, suitability_method, suitability_matrix, max_time_steps, add_tasks, add_robots, remove_robots, tasks_to_add, robots_to_add, robots_to_remove)
    end_time = time.time()
    execution_time = end_time - start_time

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used

    print(f"Simulation completed in {execution_time:.2f} seconds.")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage / (1024 * 1024)} MB")

if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    # suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    suitability_methods = ["evaluate_suitability_new"]
    # suitability_methods = ["evaluate_suitability_distance"]
    max_time_steps = 100
    add_tasks = True
    add_robots = False
    remove_robots = False

    map_file = r"test_small_open.map"
    grid = load_map(map_file) # 2D list of 0/1 representing the map
    dims = (len(grid), len(grid[0])) # dimensions of the map grid
    obstacles = create_obstacle_list(grid) # list of obstacle coordinates
    # Create the map dictionary to pass to the CBS planner
    map_dict = {
        'dimension': dims,
        'obstacles': obstacles
    }

    # print(f"MAP DICTIONARY: {map_dict}")
    for i in [5]:
        for j in [5]:
            for nc in [5]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # get initial positions list to reuse later
                            # generate_random_robot_profile now takes in the map grid and occupied positions and passes them to get_random_free_position function
                            robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(i)]
                            # changed how tasks are made here so there are no overlapping tasks
                            tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(j)]

                            suitability_matrix = calculate_suitability_matrix(robots, tasks, sm)

                            while suitability_all_zero(suitability_matrix):
                                robots = [generate_random_robot_profile_strict(f"R{i+1}", grid, set()) for i in range(i)]
                                tasks = [generate_random_task_description_strict(f"T{i+1}", grid, set(), []) for i in range(j)]
                                suitability_matrix = calculate_suitability_matrix(robots, tasks, sm)

                            output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, nc, vm)
                            cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
                            ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
                            ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
                            outputs = [output, cbba_output, ssia_output, ilp_output]
                                # param_combinations = []
                                # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
                                # with multiprocessing.Pool() as pool:
                                # pool.starmap(main_simulation, param_combinations)
                                # main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)
                            for o in outputs:
                                benchmark_simulation(o, copy.deepcopy(robots), copy.deepcopy(tasks), nc, vm, grid, map_dict, sm, suitability_matrix, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)

KeyError: 'environmental_resistance'

In [17]:
# import multiprocessing
import time
import psutil
# CBS block

def main_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    # Load the MAPF benchmark map
    map_file = r"test_small_open.map"
    #input_file = r"input.txt"
    # output_file = r"C:\Users\owner\Documents\PhD\TierLab\VBTA - Original Commented\output.txt"
    grid = load_map(map_file) # 2D list of 0/1 representing the map
    # TODO: create reuseable map dictionary from the grid so we can pass it to the CBS planner 
    # So take in the map file, and create a map dictionary with the dimensions and obstacles, 
    # then for the agents, as they are created and assigned we can update the start and goal positions
    # but unassigned robots will not be a part of the agents list until they are assigned a task because 
    # they will not be moving because they do not have a goal to reach
    dims = (len(grid), len(grid[0])) # dimensions of the map grid
    obstacles = create_obstacle_list(grid) # list of obstacle coordinates
    # Create the map dictionary to pass to the CBS planner
    map_dict = {
        'dimension': dims,
        'obstacles': obstacles
    }

    # print(f"MAP DICTIONARY: {map_dict}")
    print(f"SUITABILITY METHOD: {suitability_method}")

    # get initial positions list to reuse later
    initial_positions = set()

    # generate_random_robot_profile now takes in the map grid and occupied positions and passes them to get_random_free_position function
    robots = [generate_random_robot_profile(f"R{i+1}", grid, initial_positions) for i in range(num_robots)]
    # changed how tasks are made here so there are no overlapping tasks
    tasks = [generate_random_task_description(f"T{i+1}", grid, initial_positions, []) for i in range(num_tasks)]
    initial_robots = copy.deepcopy(robots)
    initial_tasks = copy.deepcopy(tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    
    # then propogate the initial occupied positions
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    # copy the initial positions set to use over and over
    # NOTE: occupied_positions does not need to be added to the list of obstacles because CBS will have agents avoid collisions
    occupied_positions = set(initial_positions) # use the occcupied positions as the current positions for CBS, this is just as a occupation check, not a start position

    # Calculate suitability matrix
    suitability_matrix = calculate_suitability_matrix(robots, tasks, suitability_method)
    
    # Perform task assignment using voting
    output, score, length = assign_tasks_with_voting(
        robots, tasks, suitability_matrix, num_candidates, voting_method)
    
    assigned_pairs = output[0]  # list of tuples (robot_idx, task_idx)
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True
    
    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location 
        for r in robots
        if r.assigned and r.current_task is not None
    }
    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    
    # assigned_pairs = output[0] # list of [(robot, task), (robot, task), ...] assignments
    # unassigned_robots = [robots[val].robot_id for val in output[1]]
    # unassigned_tasks = [tasks[val].task_id for val in output[2]]
    # assigned_robots = {}
    # for (robot_idx, task_idx) in assigned_pairs:
    #     assigned_robots[robots[robot_idx].robot_id] = tasks[task_idx].task_id

    
    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    # print(f"ASSIGNED PAIRS: {assigned_pairs}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    # we also run the other assignment methods here to compare their outputs and scores later
    cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
    ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
    ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
    
    # start_positions = {}
    # goal_positions = {}

    # for robot in robots:
    #     start_positions[robot.robot_id] = robot.location[0]

    # for r_id, t_id in assigned_pairs:
    #     print(f"ROBOT: {r_id}")
    #     print(f"TASK: {t_id}")
    #     # find the corresponding index for the task and robot
    #     t_index = [task.task_id for task in tasks].index(t_id)
    #     r_index = [robot.robot_id for robot in robots].index(r_id)
    #     print(f"ROBOT INDEX: {r_index}")
    #     print(f"TASK INDEX: {t_index}")
    #     # assign the current task to robot
    #     robots[r_index].current_task = tasks[t_index]
    #     # assign tasks attempted value
    #     robots[r_index].tasks_attempted = 1
    #     # assign current robot to task
    #     tasks[t_index].assigned_robot = robots[r_index]
    #     # assign the goal position with the AGENTS STRING ID not the index
    #     goal_positions[r_id] = tasks[t_index].location[0]
    #     robots[r_index].assigned = True

    # TODO: create an "agents" dictionary to pass to CBS with the start and goal positions and the agents string IDs
    # agents = []
    # for robot in robots:
    #     if robot.assigned and robot.current_task is not None:
    #         agents.append({
    #             'name': robot.robot_id,
    #             'start': start_positions[robot.robot_id],
    #             'goal': goal_positions[robot.robot_id]
    #         })
    agents = build_cbs_agents(robots, start_positions, goal_positions)

    print(f"AGENTS LIST: {agents}")

    # Create the input data dictionary for CBS, this will be passed to the CBS planner
    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    # print(f"INPUT DATA: {input_data}")

    # TODO: finish implementing the new CBS method into the simulation
    # NOTE: this will not work with the current setup, we need to pass in the
    # start and goal positions as dictionaries and a dictionary with a map representation
    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
        # could possibly fall back on the simple method here if we get a lot of issues
    else:

        # Iterate through the agents and their schedules
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            # Iterate through the schedule (list of dictionaries)
            for point in schedule:
                # Access each entry in the schedule
                # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                # Assign the paths to the robots
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    # Keep track of previous state to not run CBS when there are no changes
    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    
    # End the simulation if nothing changes for 3 timesteps (hopefully the tasks are done and the robots arent moving)
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        print(f"START POSITIONS: {start_positions}")
        print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        # before each time step, refresh the unassigned robots and tasks lists
        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        # Periodically add new tasks and robots
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        # Periodically remove robots
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1: # Otherwise will break CBS, we need at least one agent for things to run smoothly
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)
            

        # Reassign unassigned robots to unassigned tasks
        # TODO: change this to make sure newly assigned robot is more suitable for the task
        # so we will run it when there are unassigned robots AND a state change, otherwise theres no reason to keep checking
        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score
            # regenerate the master assigned robots dictionary
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }

            print(f"ROBOTS REASSIGNED")
            # print(f"NEW ASSIGNMENTS FROM REASSIGNMENT: {new_assignments}")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")
            
        
        # Update start and goal positions before cbs
        for robot in robots:
            if robot.assigned and robot.current_task:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }
        
        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            # NOTE: use occupied positions as current positions for running CBS, update goal positions whenever needed
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            agents = build_cbs_agents(robots, start_positions, goal_positions)

            print(f"AGENTS LIST: {agents}")
            
            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # if time_steps_unchanged < 3:
            #     time_steps_unchanged += 1
            #     print("No state change, skip CBS...")
            #     print(f"ALL ROBOTS: {len(robots)}")
            #     print(f"ALL TASKS: {len(tasks)}")
            #     print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            #     print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            # else:
            #     break

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    # propogate the initial occupied positions again
    initial_positions = set()
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions)
    
    assigned_pairs = cbba_output[0]
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True
        
    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                       robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
    
    else:
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            for point in schedule:
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    time_steps_unchanged = 0
    
    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        # print(f"START POSITIONS: {start_positions}")
        # print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1:
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)

        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, cbba_task_allocation, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

            # regenerate the master assigned robots dictionary based on flags
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }
            print(f"ROBOTS REASSIGNED")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")

        for robot in robots:
            if robot.assigned and robot.current_task is not None:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)
        
        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }

        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

            agents = build_cbs_agents(robots, start_positions, goal_positions)

            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

    overall_success_rate = total_success / total_tasks
    print(f"CBBA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    # propogate the initial occupied positions again
    initial_positions = set()
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions)

    assigned_pairs = ssia_output[0]
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True

    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                          robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=map_dict['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
    
    else:
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            for point in schedule:
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    time_steps_unchanged = 0
    
    for time_step in range(max_time_steps):
        # print(f"\n--- Time Step {time_step + 1} ---")
        # print(f"OCCUPIED POSITIONS: {occupied_positions}")
        # print(f"START POSITIONS: {start_positions}")
        # print(f"GOAL POSITONS: {goal_positions}")
        # print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
        # print(f"UNASSIGNED TASKS: {unassigned_tasks}")
        # # print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots]}")
        # print(f"ALL ROBOTS: {len(robots)}")
        # print(f"ALL TASKS: {len(tasks)}")
        # print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
        # print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1:
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)

        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, ssia_task_allocation, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

            # regenerate the master assigned robots dictionary based on flags
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }
            print(f"ROBOTS REASSIGNED")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")

        for robot in robots:
            if robot.assigned and robot.current_task is not None:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)
        
        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }

        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            print(f"SUITABILITY MATRIX AT TIMESTEP {time_step}: {suitability_matrix}")

            agents = build_cbs_agents(robots, start_positions, goal_positions)

            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

    overall_success_rate = total_success / total_tasks
    print(f"SSIA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}, total reassignments: {total_reassignments}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    total_reassignments = 0

    # propogate the initial occupied positions again
    initial_positions = set()
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     initial_positions.add((row, col))
    for r in robots:
        # add the robot's initial position to the occupied positions set
        initial_positions.add(r.location)

    occupied_positions = set(initial_positions)

    assigned_pairs = ilp_output[0]
    for robot_idx, task_idx in assigned_pairs:
        r = robots[robot_idx]
        t = tasks[task_idx]

        r.current_task = t
        r.assigned = True
        r.tasks_attempted = 1

        t.assigned_robot = r
        t.assigned = True
    
    # rebuild all assignment dictionaries from the robot and task flags
    assigned_robots = {r.robot_id: r.current_task.task_id for r in
                            robots if r.assigned and r.current_task is not None}
    unassigned_tasks = [t.task_id for t in tasks if not t.assigned]
    unassigned_robots = [r.robot_id for r in robots if not r.assigned]

    start_positions = {
        r.robot_id: r.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    goal_positions = {
        r.robot_id: r.current_task.location
        for r in robots
        if r.assigned and r.current_task is not None
    }

    print(f"ROBOTS: {[rob.robot_id for rob in robots]}")
    print(f"TASKS: {[tas.task_id for tas in tasks]}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    agents = build_cbs_agents(robots, start_positions, goal_positions)

    input_data = {
        'map' : {
            'dimension': map_dict['dimension'],
            'obstacles': map_dict['obstacles']
        },
        'agents': agents,
    }

    env = Environment(
        dimension=map_dict['dimension'],
        agents=input_data['agents'],
        obstacles=input_data['map']['obstacles'],
    )
    planner = CBS(env)
    solution, nodes_expanded, conflicts = planner.search()

    print(f"SOLUTION: {solution}")

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")

    else:
        for agent, schedule in solution.items():
            print(f"Schedule for {agent}:")

            for point in schedule:
                rid = [robot.robot_id for robot in robots].index(agent)
                robots[rid].current_path.append((point['x'], point['y']))
            print(f"Robot {agent} path: {robots[rid].current_path}")

    prev_state = {
        "robot_ids": set(robot.robot_id for robot in robots),
        "assigned": {robot.robot_id for robot in robots if robot.assigned},
        "unassigned": set(unassigned_robots)
    }
    time_steps_unchanged = 0

    for time_step in range(max_time_steps):

        unassigned_robots = [r.robot_id for r in robots if not r.assigned]
        unassigned_tasks = [t.task_id for t in tasks if not t.assigned]

        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, start_positions, goal_positions, 1.0, total_reward, total_success)
        if add_tasks and time_step + 1 <= 2 and random.random() < 0.5: # add tasks only in the first 2 time steps, and randomly
            print(f"ADDING NEW TASKS AT TIME STEP {time_step + 1}")
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots and time_step + 1 <= 4 and random.random() < 0.5: # add robots only in the first 4 time steps, and randomly
            print(f"ADDING NEW ROBOTS AT TIME STEP {time_step + 1}")
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)
        
        if remove_robots and time_step + 1 <= 4 and random.random() < 0.5: # remove robots only in the first 4 time steps, and randomly
            if len(assigned_robots) > 1:
                print(f"REMOVING RANDOM ROBOTS AT TIME STEP {time_step + 1}")
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions, start_positions, goal_positions)

        if unassigned_robots and unassigned_tasks:
            total_reassignments += 1
            _, unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(
                robots, tasks, num_candidates, voting_method, suitability_method, 
                unassigned_robots, unassigned_tasks, ilp_task_allocation, start_positions, goal_positions)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

            # regenerate the master assigned robots dictionary based on flags
            assigned_robots = {
                r.robot_id: r.current_task.task_id 
                for r in robots 
                if r.assigned and r.current_task is not None
            }
            print(f"ROBOTS REASSIGNED")
            print(f"ASSIGNED ROBOTS: {assigned_robots}")
            print(f"ALL ROBOTS: {[robot.robot_id for robot in robots]}")

        for robot in robots:
            if robot.assigned and robot.current_task is not None:
                start_positions[robot.robot_id] = robot.location
                goal_positions[robot.robot_id] = robot.current_task.location
            else:
                start_positions.pop(robot.robot_id, None)
                goal_positions.pop(robot.robot_id, None)

        current_state = {
            "robot_ids": set(robot.robot_id for robot in robots),
            "assigned": {robot.robot_id for robot in robots if robot.assigned},
            "unassigned": set(unassigned_robots)
        }

        if current_state != prev_state and [rob.assigned for rob in robots].count(True) >= 1:
            print("State change, re-run CBS...")
            print(f"AMOUNT OF ASSIGNED ROBOTS: {len([rob.current_task for rob in robots])}")
            print(f"ASSIGNED ROBOTS: {[rob.assigned for rob in robots].count(True)}")
            print(f"OCCUPIED POSITIONS: {occupied_positions}")
            print(f"START POSITIONS: {start_positions}")
            print(f"GOAL POSITONS: {goal_positions}")
            print(f"UNASSIGNED ROBOTS: {unassigned_robots}")
            print(f"UNASSIGNED TASKS: {unassigned_tasks}")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")
            print(f"SUITABILITY MATRIX AT TIMESTEP {time_step}: {suitability_matrix}")

            agents = build_cbs_agents(robots, start_positions, goal_positions)

            # Validation block
            start_locations = [agent['start'] for agent in agents]
            if len(start_locations) != len(set(start_locations)):
                print("ERROR: Duplicate start locations found in agent list. Aborting CBS.")
                # You can decide how to handle this. For now, we can skip the planning step for this cycle.
                # To prevent a crash, you could set solution to None and handle it.
                solution = None
            else:
                # instantiate environment and planner
                env = Environment(
                    dimension=map_dict['dimension'],
                    agents=agents,
                    obstacles=map_dict['obstacles']
                )
                planner = CBS(env)
                solution, nodes_expanded, conflicts = planner.search()
                print(f"CBS COMPLETE. New solution: {solution}")

            if solution:
                # update each robot's path based on the new CBS solution
                for robot_id, schedule in solution.items():
                    ridx = [robot.robot_id for robot in robots].index(robot_id)
                    for point in schedule:
                        # print(f"  t: {point['t']}, x: {point['x']}, y: {point['y']}")
                        # Assign the paths to the robots
                        robots[ridx].current_path.append((point['x'], point['y']))
                    # ridx = [r.robot_id for r in robots].index(robot_id)
                    # robots[ridx].current_path = schedule
                    robots[ridx].remaining_distance = len(schedule) - 1
                prev_state = current_state.copy()

            else:
                print("CBS could not find a conflict free path assignment for all agents")
                # could possibly fall back on the simple method here if we get a lot of issues
                # but for now, we will just skip CBS and continue with the simulation
                print("Skipping CBS...")
                time_steps_unchanged += 1
                if time_steps_unchanged >= 3:
                    print("No state change for 3 time steps, ending simulation.")
                    break
        else:
            print("No state change, skip CBS...")
            print(f"ALL ROBOTS: {len(robots)}")
            print(f"ALL TASKS: {len(tasks)}")
            print(f"LIST OF ALL ROBOTS: {[rob.robot_id for rob in robots]}")
            print(f"LIST OF ALL TASKS: {[tas.task_id for tas in tasks]}")

    overall_success_rate = total_success / total_tasks
    print(f"ILP: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

def benchmark_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    start_time = time.time()
    main_simulation(num_robots, num_tasks, num_candidates, voting_method, suitability_method, max_time_steps, add_tasks, add_robots, remove_robots, tasks_to_add, robots_to_add, robots_to_remove)
    end_time = time.time()
    execution_time = end_time - start_time

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used

    print(f"Simulation completed in {execution_time:.2f} seconds.")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage / (1024 * 1024)} MB")

if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    # suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    suitability_methods = ["evaluate_suitability_new"]
    # suitability_methods = ["evaluate_suitability_distance"]
    max_time_steps = 100
    add_tasks = True
    add_robots = False
    remove_robots = False
    # param_combinations = []
    for i in [5]:
        for j in [5]:
            for nc in [5]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
    # with multiprocessing.Pool() as pool:
        # pool.starmap(main_simulation, param_combinations)
                            # main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)
                            benchmark_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)

SUITABILITY METHOD: evaluate_suitability_new


AttributeError: 'list' object has no attribute 'shape'

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-560m"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# inputs = tokenizer.encode(prompt[0], return_tensors="pt")
# outputs = model.generate(inputs, max_new_tokens=600)
# print(tokenizer.decode(outputs[0]))

In [18]:
import huggingface_hub
huggingface_hub.login('***REMOVED***')

In [19]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.98s/it]


In [20]:
model = model.to("cuda")

# Function to generate text
def generate_text(prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs.input_ids, max_length=max_length, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
prompt = "Once upon a time in a land far, far away,"
print(generate_text(prompt))


Once upon a time in a land far, far away, there was a magical kingdom where all the creatures lived in harmony. But one day, a dark force threatened to destroy their peaceful way of life. The creatures banded together to defend their home, and in the process, they discovered the true meaning of courage and friendship.

In this kingdom, there lived a wise old owl named Hootie who was known for his wisdom and insight. Hootie had been watching the creatures of the kingdom for many years and had seen the dark force growing stronger by the day. He knew that something had to be done to stop it, but he didn't know what.

One day, Hootie called a meeting of all the creatures of the kingdom to discuss the problem. The creatures gathered around him, their eyes filled with fear and uncertainty. Hootie looked at them and said, "My dear friends, we are in grave danger. The dark force that threatens our kingdom is growing stronger every day. We must find a way to stop it before it's too late."

The 

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.anova import anova_lm

# Load the data
file_path = 'VBTAData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define categorical and continuous variables
categorical_factors = ['Voting_Method', 'Suitability_Method']  # Replace with actual categorical column names
continuous_factors = ['Robots', 'Tasks']   # Replace with actual continuous column names
# dependent_var = 'Score_Voting'                   # Replace with your dependent variable name
dependent_var = 'Time_Voting'                   # Replace with your dependent variable name

# Convert categorical variables to category dtype
for factor in categorical_factors:
    data[factor] = data[factor].astype('category')

# Define the formula for the ANCOVA model
formula = f"{dependent_var} ~ {' * '.join(categorical_factors)} + {' + '.join(continuous_factors)}"

# Fit the ANCOVA model
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA/ANCOVA

# Print ANCOVA results
print("Two-way ANCOVA results:")
print(anova_table)

# Post hoc analysis: Pairwise comparisons on adjusted means
print("\nPost hoc analysis (pairwise comparisons):")
for factor in categorical_factors:
    print(f"\nComparisons within {factor} (adjusted for covariates):")
    
    # Pairwise comparisons using t-tests
    mc = MultiComparison(data[dependent_var], data[factor])
    tukey_result = mc.tukeyhsd()
    
    # Display Tukey's test results for adjusted means
    print(tukey_result)

Two-way ANCOVA results:
                                        sum_sq       df            F  \
Voting_Method                     3.174761e+15      4.0   344.271446   
Suitability_Method                3.633505e+12      3.0     0.525357   
Voting_Method:Suitability_Method  5.143658e+12     12.0     0.185926   
Robots                            1.054736e+16      1.0  4575.028956   
Tasks                             1.140436e+16      1.0  4946.761087   
Residual                          4.513551e+16  19578.0          NaN   

                                         PR(>F)  
Voting_Method                     6.711993e-287  
Suitability_Method                 6.648329e-01  
Voting_Method:Suitability_Method   9.989597e-01  
Robots                             0.000000e+00  
Tasks                              0.000000e+00  
Residual                                    NaN  

Post hoc analysis (pairwise comparisons):

Comparisons within Voting_Method (adjusted for covariates):
                 

In [22]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Load the data
file_path = 'VBTAData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define column names
# group_columns = ['Score_Voting', 'Score_CBBA', 'Score_SSIA', 'Score_ILP']  # Replace with your actual column names for each group
group_columns = ['Time_Voting', 'Time_CBBA', 'Time_SSIA', 'Time_ILP']  # Replace with your actual column names for each group
covariates = ['Robots', 'Tasks']  # Replace with actual covariate column names

data = data[data['Voting_Method'] == 'rank_assignments_range']

data = data[data['Suitability_Method'] == 'evaluate_suitability_loose']

# Melt the data from wide to long format
data_long = data.melt(id_vars=covariates, value_vars=group_columns, var_name='group', value_name='dependent')

# Convert the group variable to categorical
data_long['group'] = data_long['group'].astype('category')

# Define the formula for the ANCOVA model
formula = f"dependent ~ group + {' + '.join(covariates)}"

# Fit the ANCOVA model
model = ols(formula, data=data_long).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANCOVA for better control over interactions

# Print ANCOVA results
print("ANCOVA results:")
print(anova_table)

# Check if the group factor is significant
if anova_table.loc['group', 'PR(>F)'] < 0.05:
    print("\nThe group factor is significant. Proceeding with post hoc tests.")
    
    # Post hoc analysis using Tukey's HSD
    print("\nPost hoc analysis (Tukey's HSD on adjusted means):")
    tukey_result = pairwise_tukeyhsd(data_long['dependent'], data_long['group'], alpha=0.05)
    print(tukey_result)
else:
    print("\nThe group factor is not significant. No post hoc test needed.")


ANCOVA results:
                sum_sq      df           F         PR(>F)
group     5.865748e+14     3.0  233.766498  1.620202e-139
Robots    7.722562e+14     1.0  923.297264  2.858098e-182
Tasks     5.819917e+14     1.0  695.820039  2.786051e-141
Residual  3.273714e+15  3914.0         NaN            NaN

The group factor is significant. Proceeding with post hoc tests.

Post hoc analysis (Tukey's HSD on adjusted means):
            Multiple Comparison of Means - Tukey HSD, FWER=0.05            
  group1     group2     meandiff   p-adj      lower        upper     reject
---------------------------------------------------------------------------
Time_CBBA    Time_ILP  476152.4754    0.0   349931.5305  602373.4203   True
Time_CBBA   Time_SSIA  -477618.259    0.0  -603839.2039 -351397.3141   True
Time_CBBA Time_Voting -438219.1265    0.0  -564440.0714 -311998.1816   True
 Time_ILP   Time_SSIA -953770.7344    0.0 -1079991.6793 -827549.7895   True
 Time_ILP Time_Voting -914371.6019    0.0 -1